20240111 Changes to make: 

- Stream not attributed with CDW in LP: select streams that are adjacent/share a segment with CDW habitat. 

- The Canals and Raceways layer should be used to identify canals and mark them as unregulated in extra column.   maybe we just provide all ditches and canals (unless we can sort out the “canals”) with 50 ft riparian zones and note in the meta data that field conditions would dictate whether or not such features are regulated waters. 

- C1 upstream segments were not clipped at HUC14 boundary.   

- Buffers on polygons not transitioning correctly between buffer sizes

- Check problem areas Larry identified  

In [6]:
import arcpy, os, time

In [7]:
#Set paths
home = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\RiparianZones_20240109.gdb"
newgdb = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Revision_20230809.gdb"
scratch1 = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\scratch1.gdb"
#scratch = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\scratch.gdb'
lscratch = r'Q:\WatershedNJ_KD\local_scratch.gdb'
background = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\Riparian_Zone_Input_Layers.gdb"
blayers = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\Riparian_Zone_Input_Layers.gdb"

In [ ]:
#def split_by_lines()
#def classify_split_lines()

In [ ]:
#Delete file function
#Delete field function

In [3]:
#function to split lines by a polygon and make a new field to code polygon overlap
# Default if within is NO
def split_by_polygon_zero(input_lines, splitting_polygon, field_name, output):
    split_pts = arcpy.analysis.Intersect([input_lines, splitting_polygon], os.path.join(scratch1,"temp_split_pts"), output_type = 'POINT')
    arcpy.management.SplitLineAtPoint(input_lines, split_pts, output, "5 feet")
    if len(arcpy.ListFields(output, field_name)) == 0:
         arcpy.AddField_management(output, field_name, "SHORT")
    else:
        print("Field exists")
    
    #Select lines to recode upstream field
    selection = arcpy.analysis.Intersect([output, splitting_polygon], os.path.join(scratch1, "streams_to_update_Intersect"), "ONLY_FID", None, "LINE")
    
    #Make a list from the selected features
    filename = arcpy.Describe(output).file
    list_field = f"FID_{filename}"
    oidList = [oid[0] for oid in arcpy.da.SearchCursor(selection,[list_field])]
    oidfield = arcpy.Describe(output).OIDFieldName
    fields = [oidfield, field_name]
    
    #Code features in the list 
    with arcpy.da.UpdateCursor(output, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = 0
            else:
                row[1] = 1
            curs.updateRow(row)
    del curs

In [4]:
#function to split lines by a polygon and make a new field to code polygon overlap
# Default if within is Yes
def split_by_polygon_one(input_lines, splitting_polygon, field_name, output):
    split_pts = arcpy.analysis.Intersect([input_lines, splitting_polygon], os.path.join(scratch1,"temp_split_pts"), output_type = 'POINT')
    arcpy.management.SplitLineAtPoint(input_lines, split_pts, output, "5 feet")
    if len(arcpy.ListFields(output, field_name)) == 0:
         arcpy.AddField_management(output, field_name, "SHORT")
    else:
        print("Field exists")
    
    #Select lines to recode upstream field
    selection = arcpy.analysis.Intersect([output, splitting_polygon], os.path.join(scratch1, "streams_to_update_Intersect"), "ONLY_FID", None, "LINE")
    
    #Make a list from the selected features
    filename = arcpy.Describe(output).file
    list_field = f"FID_{filename}"
    oidList = [oid[0] for oid in arcpy.da.SearchCursor(selection,[list_field])]
    oidfield = arcpy.Describe(output).OIDFieldName
    fields = [oidfield, field_name]
    
    #Code features in the list 
    with arcpy.da.UpdateCursor(output, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = 1
            else:
                row[1] = 0
            curs.updateRow(row)
    del curs

In [5]:
#function for reordering fields

def reorderfields(messy_input_file, out_path, out_name, new_field_order):
    import numpy
    arr = arcpy.da.TableToNumPyArray(messy_input_file, 
                                 new_field_order, 
                                 skip_nulls = False)
    
    #Copying messy file but just keeping OIDs and required fields. Probably better ways to do this
    #Could use another ID field instead of OID
    fieldmappings = arcpy.FieldMappings()

    # Add all fields from inputs.
    fieldmappings.addTable(messy_input_file)

    # Name fields you want. Could get these names programmatically too.
    oidfield = arcpy.Describe(messy_input_file).OIDFieldName
    keepers = [oidfield]

    # Remove all output fields you don't want.
    for field in fieldmappings.fields:
        if field.name not in keepers:
            fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))

    #Copy your file using the field mapping to select OBJECTID only. This is the destination to join the reordered columns to
    arcpy.conversion.FeatureClassToFeatureClass(messy_input_file, 
                                                out_path, 
                                                out_name,
                                                field_mapping = fieldmappings
                                               )
    #Join the sorted columns array back to your stripped down copy
    arcpy.da.ExtendTable(os.path.join(out_path, out_name), 
                         table_match_field = oidfield, 
                         in_array = arr, 
                         array_match_field = "OBJECTID")

## Optional settings

In [2]:
arcpy.env.addOutputsToMap = False

In [ ]:
#Clear project history to speed up performance
from arcpy import metadata as md

# Get the target item's Metadata object
arcpy.env.workspace = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\scratch.gdb'
scratch_files = [x for x in arcpy.ListFeatureClasses()]

for fc in scratch_files:
    tgt_item_md = md.Metadata(fc)
    tgt_item_md.deleteContent('GPHISTORY')

# Import data and identify regulated segments

In [14]:
#Feature classes of interest. Ideally link a webservice feature here to automate. 

#Landscape project critically dependent on watersource polygons
ani_polys = os.path.join(home, "CDW_all_regions")

#Landscape project mussel streams
ani_streams = r'X:\projects\njwrap\Data\NJDEP\Landscape3-3\Mussel_streams_v3_3.gdb\Envr_hab_ls_v3_3_streamhabitat'

#Natural heritage program rare plants streams and waterbodies
plant_polys = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Streams_from_Larry_Torok\CDW_Plant_waterbodies.shp'
plant_streams = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Streams_from_Larry_Torok\CDW_plant_streams.shp'

# State HUC 14s
huc14s = r'X:\projects\njwrap\Data\NJDEP\Hydr_HUC14_bnd\Hydr_HUC14_bnd.gdb\Hydr_HUC14_bnd'

# NJ 2015 NHD streams
nj2015 = os.path.join(home, "nj2015")

# NJ 2015 NHD waterbodies
nj2015_polys = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_waterbody'

#NJ 2015 LULC for wetlands
nj2020_lulc = arcpy.management.MakeFeatureLayer(r"https://services1.arcgis.com/QWdNfRs7lkPq4g4Q/arcgis/rest/services/Land_Use_2020/FeatureServer/5", "lulc2020")

In [6]:
# NJ 2015 NHD streams
if arcpy.Exists(os.path.join(home, "nj2015")):
    print("nhd 2015 copy exists")
    nj2015 = os.path.join(home, "nj2015")
else:
    print("Copying nhd 2015")
    nj2015_streams = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_streams'
    #Copy flowlines out of the topology so that it can be more easily manipulated
    arcpy.management.CopyFeatures(nj2015_streams, os.path.join(home, "nj2015"))

nhd 2015 copy exists


In [7]:
#E&T species critically dependent on the watercourse, from Landscape Project
if arcpy.Exists(os.path.join(home, "CDW_all_regions")):
    print("LP all exists")
    ani_polys = os.path.join(home, "CDW_all_regions")
else: 
    print("Run the following Landscape Project CDW polygons code")                               

LP all exists


In [8]:
#Align swqs streams to nhd streams
if arcpy.Exists(r"X:\projects\njwrap\Data\NJDEP\NJ_surface_water_categories\Hydr_water_stream_swqs.gdb\Hydr_water_stream_swqs_aligned"):
    print("swqs aligned exists")
    swqs = r"X:\projects\njwrap\Data\NJDEP\NJ_surface_water_categories\Hydr_water_stream_swqs.gdb\Hydr_water_stream_swqs_aligned"
else: 
    print("swqs needs to be aligned")
    swqs1 = r'X:\projects\njwrap\Data\NJDEP\NJ_surface_water_categories\Hydr_water_stream_swqs.gdb\Hydr_water_stream_swqs'
    arcpy.edit.AlignFeatures(
        in_features= swqs1,
        target_features= nj2015,
        search_distance= "60 Feet",
        match_fields=None
    ) #1 hour 45 min

swqs aligned exists


## Landscape Project CDW polygons

In [ ]:
if arcpy.Exists(os.path.join(newgdb, 'Landscape_proj_all_cdw')):
    print("LP all exists")
 
#Assemble Landscape Project data stored in separate gdbs (probably could have done a dictionary)
arcpy.env.workspace = r'X:\projects\njwrap\Data\NJDEP\Landscape3-3\Region_data'

# List all file geodatabases in the current workspace
lp_gdbs = arcpy.ListWorkspaces("*", "FileGDB")  

lp_list = []
for lp in lp_gdbs:
    arcpy.env.workspace = lp
    for fc in arcpy.ListFeatureClasses('E*'):
        fc_path = os.path.join(arcpy.env.workspace, fc)
        print(fc_path)
        lp_list.append(fc_path)

lp_list

In [ ]:
#Assemble Landscape Project data stored in separate gdbs (probably could have done a dictionary)
#https://community.esri.com/t5/arcgis-pro-questions/arcgis-pro-join-one-to-many-join-by-attribute/td-p/589401/page/2
arcpy.env.workspace = r'X:\projects\njwrap\Data\NJDEP\Landscape3-3\Region_data'

spnum = 0

for lp in lp_gdbs:
    arcpy.env.workspace = lp
    #copy polygons for permanent join
    for fc in arcpy.ListFeatureClasses('E*'):
        fc_path = os.path.join(arcpy.env.workspace, fc)
        fc_copy = arcpy.management.CopyFeatures(fc_path, "fc_copy")
    for tbl in arcpy.ListTables('*_sp*'):
        tbl_path = os.path.join(arcpy.env.workspace, tbl)
    # list the polygon feature class and table that want to be joined
    tableList = [fc_copy, tbl_path]
    # define the query for matching
    whereclause = f"fc_copy.LINKID = {tbl}.LINKID"
    # name the temporary layer name created by MakeQueryTable
    lyrName = "sp_layer"
    # name the output fc name
    region_name = lp_list[spnum].split("_")[-1]
    print(region_name)
    out_FeatureClass = os.path.join(scratch, f"Envr_hab_ls_v3_3_{region_name}_sp")
    print(out_FeatureClass)   
    arcpy.MakeQueryTable_management(tableList, lyrName,"USE_KEY_FIELDS", "", "", whereclause)
    arcpy.CopyFeatures_management(lyrName, out_FeatureClass)
    if spnum == 0:
        LP_all = arcpy.management.CreateFeatureclass(out_path = newgdb, out_name = 'Landscape_proj_all_cdw', geometry_type = 'POLYGON', template = out_FeatureClass, spatial_reference = out_FeatureClass)#empty dataset to store each region data
        arcpy.management.Append(out_FeatureClass, LP_all)
        spnum += 1
        print(lp)
        continue
    else: 
        arcpy.management.Append(out_FeatureClass, LP_all)
        spnum += 1
        print(lp) 

In [46]:
LP_all = os.path.join(newgdb, "Landscape_proj_all_cdw")
LP_sel = arcpy.management.MakeFeatureLayer(LP_all, "LP_lyr", where_clause = "FHA_SCDW = 'Yes'")

In [4]:
nj2015_polys = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_waterbody'

arcpy.management.MakeFeatureLayer(nj2015_polys, "streams", where_clause = "FTYPE_DESCRIPTION ='Stream/River' Or FTYPE_DESCRIPTION = 'Canal/Ditch' ")
arcpy.management.SelectLayerByLocation("streams", "BOUNDARY_TOUCHES", LP_sel)

<Result 'streams'>

In [5]:
#Make a feature class of the selected waterbodies.  
arcpy.conversion.ExportFeatures("streams", os.path.join(home, "ET_streams_non_Del"), where_clause = "GNIS_NAME <> 'Delaware River'")
arcpy.conversion.ExportFeatures("streams", os.path.join(home, "ET_streams_DR"), where_clause = "GNIS_NAME = 'Delaware River'")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\ET_streams_DR'>

In [6]:
#Chop up the polygons that are too long and thus are over-inclusive of stream stretches
arcpy.management.SubdividePolygon(
    in_polygons =os.path.join(home, "ET_streams_non_Del"),
    out_feature_class = os.path.join(home, "ET_non_Del_SubdividePolygon"),
    method="NUMBER_OF_EQUAL_PARTS",
    num_areas=10,
    target_area=None,
    target_width=None,
    split_angle=0,
    subdivision_type="STACKED_BLOCKS"
)

arcpy.management.SubdividePolygon(
    in_polygons =os.path.join(home, "ET_streams_DR"),
    out_feature_class = os.path.join(home, "ET_DR_SubdividePolygon"),
    method="NUMBER_OF_EQUAL_PARTS",
    num_areas=25,
    target_area=None,
    target_width=None,
    split_angle=0,
    subdivision_type="STACKED_BLOCKS"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\ET_DR_SubdividePolygon'>

In [7]:
arcpy.management.MultipartToSinglepart(
    in_features= os.path.join(home, "ET_non_Del_SubdividePolygon"),
    out_feature_class= os.path.join(home, "ET_nonDel_MultipartToSinglepart")
)

arcpy.management.MultipartToSinglepart(
    in_features= os.path.join(home, "ET_DR_SubdividePolygon"),
    out_feature_class= os.path.join(home, "ET_DR_MultipartToSinglepart")
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\ET_DR_MultipartToSinglepart'>

In [8]:
arcpy.management.Merge([os.path.join(home, "ET_nonDel_MultipartToSinglepart"), os.path.join(home, "ET_DR_MultipartToSinglepart")],
                      os.path.join(home, "ET_s_MultipartToSinglepart"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\ET_s_MultipartToSinglepart'>

In [12]:
arcpy.management.MakeFeatureLayer(os.path.join(home, "ET_s_MultipartToSinglepart"), "s_singlepart")
arcpy.management.SelectLayerByLocation("s_singlepart", "INTERSECT", LP_sel)

<Result 's_singlepart'>

In [13]:
arcpy.management.CopyFeatures("s_singlepart", os.path.join(home,"s_selection"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\s_selection'>

In [20]:
#Select lakes/ponds/etc. adjacent to E&T
#arcpy.management.MakeFeatureLayer(nj2015_polys, "wb", where_clause = "FTYPE_DESCRIPTION <>'Stream/River' And FTYPE_DESCRIPTION <> 'Canal/Ditch' ")
#arcpy.management.SelectLayerByLocation("wb", "BOUNDARY_TOUCHES", LP_sel)
#arcpy.management.CopyFeatures("wb", os.path.join(home,"wb_selection"))


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\wb_selection'>

In [47]:
#os.path.join(home,"wb_selection") Excluding for now
arcpy.management.Merge([LP_sel, os.path.join(home,"s_selection")], os.path.join(home, "fhascdw_and_streams"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\fhascdw_and_streams'>

In [48]:
arcpy.management.Dissolve(os.path.join(home, "fhascdw_and_streams"), 
                                    os.path.join(home, "CDW_all_regions"), 
                                    None, 
                                    None, 
                                    "SINGLE_PART", 
                                    "DISSOLVE_LINES", '')

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\CDW_all_regions'>

In [49]:
#Output of this process:
ani_polys = os.path.join(home, "CDW_all_regions")

1/12 Notes on potential problems with this selection:
- waterbodies are selected even with very small overlap with E&T. Do we want to set a minimum % or other type of contact requirement?
- Some Delaware not selected if not touching, probably due to differences between datasets and not a real intention for them not to touch. Could select "within a distance"
- How does this look? Too inclusive, not inclusive enough, just right?


## Trace Network Clean Up

- Repair geometry
- Remove coastlines
- Identify dirty areas
- Manually inspect for self-intersections and delete vertices as necessary

## Transferring SWQS classes to nhd lines

In [ ]:
#copy swqs lines
#select and attribute lines with the selection above so that nulls have all zeroes
arcpy.management.CopyFeatures(swqs, os.path.join(home, "swqs_copy"))
# Delete null CATEGORY row

In [82]:
#Select higher regulated segments. Buffer SWQS and and transfer category from selected streams to buffer

# Export segments with higher regulations C1, TP, TM
swqs_sel = os.path.join(home, "swqs_copy")
#arcpy.conversion.ExportFeatures(swqs, os.path.join(home, "swqs_selection"), "(CATEGORY LIKE '%FW%' Or CATEGORY LIKE '%PL%') And (CATEGORY LIKE '%C1%' Or CATEGORY LIKE '%TP%' Or CATEGORY LIKE '%TM%')")

# Add field to NJ Streams for each type swqs
if len(arcpy.ListFields(swqs_sel, 'C1')) == 0:
         arcpy.AddField_management(swqs_sel, "C1", "SHORT")
else:
    print("Field exists")
if len(arcpy.ListFields(swqs_sel, 'TP')) == 0:
         arcpy.AddField_management(swqs_sel, "TP", "SHORT")
else:
    print("Field exists")
if len(arcpy.ListFields(swqs_sel, 'TM')) == 0:
         arcpy.AddField_management(swqs_sel, "TM", "SHORT")
else:
    print("Field exists")

#Parse codes to separate columns from swqs
fc = swqs_sel
fields = ['CATEGORY', 'C1', 'TP', 'TM']

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if ('FW' in row[0] or 'PL' in row[0]):
            if 'C1' in row[0]:
                row[1] = 1
            else:
                row[1] = 0
        else: 
            row[1] = 0
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if ('FW' in row[0] or 'PL' in row[0]):
            if 'TP' in row[0]:
                row[2] = 1
            else:
                row[2] = 0
        else: 
            row[2] = 0
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        if ('FW' in row[0] or 'PL' in row[0]):
            if 'TM' in row[0]:
                row[3] = 1
            else:
                row[3] = 0
        else: 
            row[3] = 0
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

Field exists
Field exists
Field exists


In [83]:
#Calculate transfer field on swqs lines "Transfer_code"
new_field = "Transfer_code"

if len(arcpy.ListFields(swqs_sel, new_field)) == 0:
         arcpy.AddField_management(swqs_sel, new_field, "SHORT")
else:
    print("Field exists")

fields = ["C1", "TP", "TM", new_field]

# Create update cursor for feature class 
with arcpy.da.UpdateCursor(swqs_sel, fields) as cursor:
    for row in cursor:
        if (row[0] == 0 and row[2] == 1):
            row[3] = 1
        elif row[0] == 1 and row[1] == 1:
            row[3] = 2
        elif row[0] == 1 and row [2] == 1:
            row[3] = 3
        elif row[0] == 1 and row[1] == 1:
            row[3] = 4
        elif row[0] == 1 and row[1] == 0 and row[2] == 0:
            row[3] = 5
        else:
            row[3] = 0

        # Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

Field exists


In [7]:
#Dissolve by category to cut down on individual line segments that all get the same buffer
swqs_sel = os.path.join(home, "swqs_copy")
arcpy.management.Dissolve(in_features = swqs_sel, 
                          out_feature_class = os.path.join(home,"swqs_select_diss_single"),
                          dissolve_field = ['C1', 'TP', 'TM',"Transfer_code"], 
                          statistics_fields = "",
                          multi_part = "MULTI_PART",
                          unsplit_lines = "DISSOLVE_LINES")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\swqs_select_diss_single'>

In [18]:
#Remove coastlines from flowlines
no_coast = arcpy.management.MakeFeatureLayer(nj2015, 
                                             "no_coast", 
                                             where_clause = "FCODE <> 56600")

In [19]:
#Dissolve nhd streams for a cleaner transfer
nhd_diss = arcpy.management.Dissolve(no_coast, os.path.join(home,"nj2015_diss"), 
                                     "GNIS_NAME", 
                                     "",
                                     "SINGLE_PART",
                                     "UNSPLIT_LINES")

In [11]:
# Loading variables without rerunning
nhd_diss = os.path.join(home,"nj2015_diss")

In [32]:
#make end points and start points
swqs_ends = arcpy.management.FeatureVerticesToPoints(os.path.join(home,"swqs_select_diss_single"), os.path.join(scratch1,"swqs_end_pts"), 'BOTH_ENDS')
#delete identical points
swqs_ends = arcpy.management.DeleteIdentical(os.path.join(scratch1, "swqs_end_pts"), "Shape", "1 foot")


In [33]:
swqs_ends = os.path.join(scratch1, "swqs_end_pts")

#Field mapping
#  https://help.arcgis.com/en/arcgisdesktop/10.0/help/index.html#//00080000000q000000
fieldmappings = arcpy.FieldMappings()

# Add all fields from inputs.
fieldmappings.addTable(swqs_ends)
fieldmappings.addTable(os.path.join(home,"swqs_select_diss_single"))

# Name fields you want. Could get these names programmatically too.
featureclass = swqs_ends
keepers = [f.name for f in arcpy.ListFields(featureclass)]
keepers.append("Transfer_code")

# Remove all output fields you don't want.
for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))


TransferFieldIndex = fieldmappings.findFieldMapIndex("Transfer_code")
fieldmap = fieldmappings.getFieldMap(TransferFieldIndex)
fieldmap.mergeRule = "count" #If there is more than one value, this is a transition point
fieldmappings.replaceFieldMap(TransferFieldIndex, fieldmap)

In [34]:
#Only keep points that overlap transitions between two different swqs types
#Spatial join points and swqs to see where points overlap changes in classifications. Those are the only points we need to retain
arcpy.analysis.SpatialJoin(swqs_ends, 
                           os.path.join(home,"swqs_select_diss_single"), 
                           os.path.join(scratch1, "swqs_split_pts_SpatialJoin"), 
                           "JOIN_ONE_TO_ONE", 
                           "KEEP_ALL", 
                           field_mapping = fieldmappings,
                           match_option = "INTERSECT"
                          )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\swqs_split_pts_SpatialJoin'>

In [35]:
#Select points at transitions between swqs codes
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "swqs_split_pts_SpatialJoin"), "overlap_pts", where_clause = "Join_Count > 1")

<Result 'overlap_pts'>

In [40]:
arcpy.edit.Snap(
    in_features="overlap_pts",
    snap_environment=f"{nhd_diss} EDGE '100 Feet'"
)

<Result 'overlap_pts'>

In [41]:
arcpy.management.CopyFeatures("overlap_pts", os.path.join(home,"overlap_pts"))
#Some hand editing 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\overlap_pts'>

In [54]:
#split nhd by points where classifications change in the middle of a segment rather than at confluences
arcpy.management.SplitLineAtPoint(nhd_diss, os.path.join(home,"overlap_pts"), os.path.join(home, "nhd_swqs_split1"), "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\nhd_swqs_split1'>

In [55]:
# Loading variables without rerunning
swqs_sel_diss = arcpy.management.MakeFeatureLayer(os.path.join(home,"swqs_select_diss_single"), "selected_swqs", where_clause = "Transfer_code <> 0")

In [56]:
#Field mapping
#  https://help.arcgis.com/en/arcgisdesktop/10.0/help/index.html#//00080000000q000000
fieldmappings = arcpy.FieldMappings()

# Add all fields from inputs.
fieldmappings.addTable(os.path.join(home, "nhd_swqs_split1"))
fieldmappings.addTable(swqs_sel_diss)

# Name fields you want. Could get these names programmatically too.
featureclass = os.path.join(home, "nhd_swqs_split1")
keepers = [f.name for f in arcpy.ListFields(featureclass)]
keepers.extend(["C1", "TP", "TM", "Transfer_code"])

# Remove all output fields you don't want.
for field in fieldmappings.fields:
    if field.name not in keepers:
        fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field.name))


In [57]:
#spatial join
arcpy.analysis.SpatialJoin(os.path.join(home,"nhd_swqs_split1"),
                                         swqs_sel_diss,
                                         os.path.join(home,"nhd_swqs_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         join_type = "KEEP_ALL", 
                                         field_mapping = fieldmappings, 
                                         match_option = "SHARE_A_LINE_SEGMENT_WITH")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\nhd_swqs_join'>

Some missed  classifications lingering in coastal areas. Not very bad problem. Can be fixed within polygons. Perhaps use SWQS copy to attribute waterbodies later on. Perhaps clip at coast and sub in SWQS lines directly. 

In [58]:
nj2015_swqs = os.path.join(home,"nhd_swqs_join")
arcpy.management.RepairGeometry(nj2015_swqs, "DELETE_NULL", "ESRI")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\nhd_swqs_join'>

## Split NJ 2015 streamlines by Threatened and Endangered species lines and polys

### Add mussel and plant stream lines

In [8]:
nj2015_swqs = os.path.join(home,"nhd_swqs_join")

In [44]:
if arcpy.Exists(os.path.join(scratch1,"cdw_temp_linesA")):
    print("File exists. It takes 3 hours to align it, so don't overwrite!")
else:
    inputs = [ani_streams, plant_streams]
    output = os.path.join(scratch1,"cdw_streams")
    arcpy.management.Merge(inputs, output)
    arcpy.management.Dissolve(output, 
                              os.path.join(scratch1,"cdw_temp_linesA"),
                              multi_part = "SINGLE_PART",
                              unsplit_lines = "UNSPLIT_LINES" )

In [45]:
# 3 hour run
arcpy.edit.AlignFeatures(
        in_features= os.path.join(scratch1,"cdw_temp_linesA"),
        target_features= nj2015_swqs,
        search_distance= "60 Feet",
        match_fields=None
    )
#Didn't work that well. Maybe just skip and make end points and snap them to stream lines

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_temp_linesA'>

In [67]:
input_fc = os.path.join(scratch1,"cdw_temp_linesA")
buffer2 = arcpy.analysis.Buffer(input_fc, os.path.join(scratch1,"temp_buffer2"), "80 Feet", "FULL", "FLAT", "ALL", None, "PLANAR")                        

In [69]:
arcpy.analysis.Clip(
    in_features=os.path.join(scratch1,"cdw_temp_linesA"),
    clip_features=buffer2,
    out_feature_class= os.path.join(scratch1,"cdw_temp_linesA_Clip")
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_temp_linesA_Clip'>

In [68]:
#Create split points
arcpy.analysis.Intersect(
    in_features= [buffer2, nj2015_swqs],
    out_feature_class=os.path.join(home,"cdw_swqs_intersect"),
    join_attributes="ONLY_FID",
    cluster_tolerance=None,
    output_type="POINT"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\cdw_swqs_intersect'>

In [72]:
#split streams by cdw streams
nj2015_temp1 = arcpy.management.SplitLineAtPoint(nj2015_swqs, 
                                                 os.path.join(home,"cdw_swqs_intersect"), 
                                                 os.path.join(home,"nhd_cdw_split"), "5 feet")

In [73]:
arcpy.management.MakeFeatureLayer(nj2015_temp1, "cdw_lines1")
arcpy.management.SelectLayerByLocation("cdw_lines1", "SHARE_A_LINE_SEGMENT_WITH", "cdw_temp_linesA_Clip")
arcpy.management.SelectLayerByLocation("cdw_lines1", 
                                       "WITHIN", 
                                       buffer2, 
                                       selection_type = "SUBSET_SELECTION")

<Result 'cdw_lines1'>

In [74]:
input_fc = nj2015_temp1
field_name = "all_cdw"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("cdw_lines1",["OID@"])]
oidfield = arcpy.Describe("cdw_lines1").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[0] = 0 
        curs.updateRow(row)
del curs

### Add cdw plant and animal polys

In [75]:
#Combine 2 CDW sets of polygons into one set
in_features = [ani_polys, plant_polys]
arcpy.management.Merge(in_features, os.path.join(scratch1,"all_cdw_polys"), "ONLY_FID")
cdw_polys = arcpy.management.Dissolve(os.path.join(scratch1,"all_cdw_polys"), 
                                      os.path.join(scratch1,"all_cdw_polys_diss"), 
                                      "", 
                                      "", 
                                      "SINGLE_PART")

In [ ]:
#Split lines by cdw polygons and recode
input_lines = nj2015_temp1
splitting_polygon = cdw_polys
field_name = "all_cdw"
output =  os.path.join(home,"nhd_cdw_split1")

split_by_polygon_one(input_lines, splitting_polygon, field_name, output)

#### A long version of this function in case it doesn't work

In [78]:
#Create split points
arcpy.analysis.Intersect(
    in_features= [cdw_polys, nj2015_temp1],
    out_feature_class= os.path.join(scratch1,"temp_pts1"),
    join_attributes="ONLY_FID",
    cluster_tolerance=None,
    output_type="POINT"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\temp_pts1'>

In [80]:
#split streams by cdw streams
nj2015_temp2 = arcpy.management.SplitLineAtPoint(nj2015_temp1, os.path.join(scratch1,"temp_pts1"), os.path.join(home,"nhd_cdw_split1"), "5 feet")

In [81]:
arcpy.management.MakeFeatureLayer(nj2015_temp2, "cdw_lines2")
arcpy.management.SelectLayerByLocation("cdw_lines2", "WITHIN", cdw_polys)

<Result 'cdw_lines2'>

In [82]:
input_fc = nj2015_temp2
field_name = "all_cdw"

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("cdw_lines2",["OID@"])]
oidfield = arcpy.Describe("cdw_lines2").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        curs.updateRow(row)
del curs

## Excluding unregulated streams
See table: https://rutgersconnect-my.sharepoint.com/:w:/r/personal/kdd56_crssa_rutgers_edu/Documents/NJWRAP/NJWRAP_Task2_shared_documents/Regulatory_Riparian_Areas/20230912_Riparian_regulatory_buffer_report_revised.docx?d=wf10f0e61df20477eb0c6270674056ca0&csf=1&web=1&e=tgsIj3
Still do this at the top of the project because it affects upstream selections

In [83]:
#Convert null values to 0
fieldNames = ['all_cdw', 'C1', 'TP', 'TM']
fieldCount = len(fieldNames)

with arcpy.da.UpdateCursor(nj2015_temp2, fieldNames) as curU:  
    for row in curU:  
        rowU = row  
        for field in range(fieldCount):  
            if rowU[field] == None:  
                rowU[field] = 0
            elif rowU[field] == "":
                rowU[field] = 0
            curU.updateRow(rowU)
del curU

### Labeling unregulated LULC classes

In [43]:
nj2015_temp2 = os.path.join(home,"nhd_CDW_split1")

In [44]:
#LULC excluded classes--identify them so that there is no upstream trace from theres

#5120 (Canals) 
#7310, 7320, 7330 (Stone Quarries, Sand and Gravel Pits (Borrow Pits), Other Mining)
#5430 (Atlantic Ocean) 
#5420 (Dredged Lagoon)
#6111      Saline Marsh (Low marsh) 
#6112      Saline Marsh (High marsh) 
#6130      Vegetated Dune Communities 
#6141      Phragmites Dominate Coastal Wetlands 


nj2020_lulc = arcpy.management.MakeFeatureLayer(r"https://services1.arcgis.com/QWdNfRs7lkPq4g4Q/arcgis/rest/services/Land_Use_2020/FeatureServer/5", "lulc2020")
splitting_polygon1 = arcpy.management.MakeFeatureLayer(lulc, 
                                                       "lulc_unregulated", 
                                                       where_clause = "LU15 = 5120 Or LU15 = 7310 Or LU15 = 7320 Or LU15 = 7330 Or LU15 = 5430 Or LU15 = 6111 Or LU15 = 6112 Or LU15 = 6130 OR LU15 = 6141"
                                                       )

arcpy.management.Dissolve("lulc_unregulated", os.path.join(home,"lulc_exclusions_diss"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\lulc_exclusions_diss'>

In [47]:
#Add exclusion field for LULC
input_lines = nj2015_temp2
splitting_polygon = os.path.join(home,"lulc_exclusions_diss")
field_name = "regulated"
output =  os.path.join(home, "stream_regs_with_exclusions_v1")

split_by_polygon_zero(input_lines, splitting_polygon, field_name, output)

### Labeling canals

In [94]:
#Add canals and raceways
arcpy.management.MakeFeatureLayer(r"https://mapsdep.nj.gov/arcgis/rest/services/Features/Hydrography/MapServer/45", 
                                  "canals", 
                                  where_clause = "CONDITION = 'Water'")

<Result 'canals'>

In [10]:
# How to select and include the Canals
arcpy.topographic.PolygonToCenterline(
    in_features="canals",
    out_feature_class= os.path.join(scratch1, "canals_PolygonToCenterline"),
    connecting_features=None
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\canals_PolygonToCenterline'>

In [15]:
arcpy.edit.AlignFeatures(
        in_features= os.path.join(scratch1, "canals_PolygonToCenterline"),
        target_features= nj2015,
        search_distance= "60 Feet",
        match_fields=None
    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\canals_PolygonToCenterline'>

In [19]:
buffer2 = arcpy.analysis.Buffer(os.path.join(scratch1, "canals_PolygonToCenterline"), os.path.join(scratch1,"temp_buffer2"), "80 Feet", "FULL", "FLAT", "ALL", None, "PLANAR")                        

In [20]:
arcpy.analysis.Clip(
    in_features= os.path.join(scratch1, "canals_PolygonToCenterline"),
    clip_features=buffer2,
    out_feature_class= os.path.join(scratch1, "canals_Centerline_clip")
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\canals_Centerline_clip'>

In [21]:
#Create split points
arcpy.analysis.Intersect(
    in_features= [buffer2, nj2015],
    out_feature_class=os.path.join(home,"canals_swqs_intersect"),
    join_attributes="ONLY_FID",
    cluster_tolerance=None,
    output_type="POINT"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\canals_swqs_intersect'>

In [25]:
arcpy.management.MakeFeatureLayer(os.path.join(home,"canals_swqs_intersect"), "pts_lyr")
arcpy.management.SelectLayerByLocation("pts_lyr", "INTERSECT", os.path.join(scratch1, "canals_Centerline_clip"))

<Result 'pts_lyr'>

In [26]:
arcpy.management.CopyFeatures("pts_lyr", os.path.join(scratch1,"pts_lyr_sel"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\pts_lyr_sel'>

In [48]:
#split streams by cdw streams
arcpy.management.SplitLineAtPoint(os.path.join(home, "stream_regs_with_exclusions_v1"), 
                                  os.path.join(scratch1,"pts_lyr_sel"), 
                                  os.path.join(home, "stream_regs_with_exclusions_v2"), "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\stream_regs_with_exclusions_v2'>

In [49]:
arcpy.management.MakeFeatureLayer(os.path.join(home, "stream_regs_with_exclusions_v2"), "lyr_lines1")
#arcpy.management.SelectLayerByLocation("lyr_lines1", "SHARE_A_LINE_SEGMENT_WITH", os.path.join(scratch1, "canals_Centerline_clip"))
arcpy.management.SelectLayerByLocation("lyr_lines1", 
                                       "WITHIN", 
                                       buffer2)
arcpy.management.SelectLayerByAttribute("lyr_lines1",
                                        selection_type = "ADD_TO_SELECTION",
                                        where_clause = "GNIS_NAME LIKE '%Canal'")

<Result 'lyr_lines1'>

In [50]:
input_fc = os.path.join(home, "stream_regs_with_exclusions_v2")
field_name = "regulated"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("lyr_lines1",["OID@"])]
oidfield = arcpy.Describe("lyr_lines1").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 0
        else:
            row[0] = 1 
        curs.updateRow(row)
del curs

Field exists


In [51]:
input_fc = os.path.join(home, "stream_regs_with_exclusions_v2")
field_name = "Canal"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("lyr_lines1",["OID@"])]
oidfield = arcpy.Describe("lyr_lines1").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[0] = 0 
        curs.updateRow(row)
del curs

# Select upstream reaches for each category

In [ ]:
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")

## Create trace network for TP/C1

In [ ]:
#Create feature dataset for trace network
#arcpy.management.CreateFeatureDataset(r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\20221104_Regulatory_buffers.gdb", "combined_lines", nj2015_merge)
#arcpy.env.workspace = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\20221104_Regulatory_buffers.gdb\combined_lines"
#arcpy.conversion.ExportFeatures(r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_streams', "nj2015a")
arcpy.conversion.ExportFeatures(r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\20221104_Regulatory_buffers.gdb\topology_test\nj2015_merge_diss1", "nj2015_merge_diss1_copy")


In [ ]:
#Clean up steps

#Select by location,Intersection with Atlantic ocean from NHD Waterbodies to clean up unneded
#arcpy.management.SelectLayerByAttribute(nj2015_polys, "NEW_SELECTION", "FCODE = 49300 Or FCODE = 44500", None)
#arcpy.management.SelectLayerByLocation("nj2015_merge_diss1_copy", "INTERSECT", nj2015_polys, None, "ADD_TO_SELECTION", "NOT_INVERT")
#arcpy.management.DeleteRows("nj2015_merge_diss1_copy")
arcpy.management.SelectLayerByAttribute("nj2015_merge_diss1_copy", "NEW_SELECTION", "FCODE = 56600", None
arcpy.management.RepairGeometry("nj2015_merge_diss1_copy")

In [ ]:
#Set workspace and create variable for new copy of lines
arcpy.env.workspace = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\20221104_Regulatory_buffers.gdb\combined_lines"
nj2015_merge = "nj2015_merge_diss1_copy"

In [ ]:
#Create trace network from merged lines with TP start points as input junctions
#The fact that this is needed is concerting. When I ran topology checks there were no overlaps...

in_feature_dataset = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\20221104_Regulatory_buffers.gdb\combined_lines"
#input_junctions = "TP_start_pts"
#arcpy.management.RepairGeometry(nj2015_merge)
input_edges = nj2015_merge

arcpy.tn.CreateTraceNetwork(in_feature_dataset, "nj_merge_network","",[[input_edges, 'SIMPLE_EDGE']])

In [ ]:
#arcpy.tn.EnableNetworkTopology("nj_TP_merge_network", "", "ONLY_ERRORS")
#arcpy.management.RepairGeometry("nj2015a")
#arcpy.topographic.RepairSelfIntersection("nj2015_merge_diss1_copy", "SPLIT", None, "IGNORE_ENDS")
#arcpy.tn.EnableNetworkTopology("nj_merge_network")
arcpy.tn.ValidateNetworkTopology("nj_merge_network")

In [ ]:
#Copy network streams to put all changes on rather than the network itselft
arcpy.conversion.ExportFeatures("streams_from_featureservice", r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\streams_from_fs_copy",
                                '', "NOT_USE_ALIAS", 
                                'COMID "ComID" true true false 4 Long 0 0,First,#,streams_from_featureservice,COMID,-1,-1;PERMANENT_IDENTIFIER "PERMANENT_IDENTIFIER" true true false 40 Text 0 0,First,#,streams_from_featureservice,PERMANENT_IDENTIFIER,0,40;FDATE "FDATE" true true false 8 Date 0 0,First,#,streams_from_featureservice,FDATE,-1,-1;RESOLUTION "Resolution" true true false 4 Long 0 0,First,#,streams_from_featureservice,RESOLUTION,-1,-1;GNIS_ID "GNIS_ID" true true false 10 Text 0 0,First,#,streams_from_featureservice,GNIS_ID,0,10;GNIS_NAME "GNIS_NAME" true true false 65 Text 0 0,First,#,streams_from_featureservice,GNIS_NAME,0,65;LENGTHKM "LENGTHKM" true true false 8 Double 0 0,First,#,streams_from_featureservice,LENGTHKM,-1,-1;REACHCODE "REACHCODE" true true false 14 Text 0 0,First,#,streams_from_featureservice,REACHCODE,0,14;FLOWDIR "FlowDir" true true false 4 Long 0 0,First,#,streams_from_featureservice,FLOWDIR,-1,-1;WBAREACOMID "WBAREACOMID" true true false 4 Long 0 0,First,#,streams_from_featureservice,WBAREACOMID,-1,-1;WBAREA_PERMANENT_IDENTIFIER "WBAREA_PERMANENT_IDENTIFIER" true true false 40 Text 0 0,First,#,streams_from_featureservice,WBAREA_PERMANENT_IDENTIFIER,0,40;FTYPE "FType" true true false 4 Long 0 0,First,#,streams_from_featureservice,FTYPE,-1,-1;FCODE "FCode" true true false 4 Long 0 0,First,#,streams_from_featureservice,FCODE,-1,-1;ENABLED "Enabled" true true false 2 Short 0 0,First,#,streams_from_featureservice,ENABLED,-1,-1;STREAMLEVEL "STREAMLEVEL" true true false 4 Long 0 0,First,#,streams_from_featureservice,STREAMLEVEL,-1,-1;STREAMORDER "STREAMORDER" true true false 4 Long 0 0,First,#,streams_from_featureservice,STREAMORDER,-1,-1;FTYPE_DESCRIPTION "FTYPE_DESCRIPTION" true true false 50 Text 0 0,First,#,streams_from_featureservice,FTYPE_DESCRIPTION,0,50;FCODE_DESCRIPTION "FCODE_DESCRIPTION" true true false 255 Text 0 0,First,#,streams_from_featureservice,FCODE_DESCRIPTION,0,255;HU8 "HU8" true true false 8 Text 0 0,First,#,streams_from_featureservice,HU8,0,8;CREATED_USER "CREATED_USER" true true false 255 Text 0 0,First,#,streams_from_featureservice,CREATED_USER,0,255;CREATED_DATE "CREATED_DATE" true true false 8 Date 0 0,First,#,streams_from_featureservice,CREATED_DATE,-1,-1;LAST_EDITED_USER "LAST_EDITED_USER" true true false 255 Text 0 0,First,#,streams_from_featureservice,LAST_EDITED_USER,0,255;LAST_EDITED_DATE "LAST_EDITED_DATE" true true false 8 Date 0 0,First,#,streams_from_featureservice,LAST_EDITED_DATE,-1,-1;GLOBALID "GLOBALID" false false true 38 GlobalID 0 0,First,#,streams_from_featureservice,GLOBALID,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,streams_from_featureservice,Shape_Length,-1,-1;FLOWDIRECTION "Flow direction" true true false 2 Short 0 0,First,#,streams_from_featureservice,FLOWDIRECTION,-1,-1;FLOWDIRECTION_1 "Flow direction" true true false 2 Short 0 0,First,#,streams_from_featureservice,FLOWDIRECTION_1,-1,-1;FLOWDIRECTION_2 "Flow direction" true true false 2 Short 0 0,First,#,streams_from_featureservice,FLOWDIRECTION_2,-1,-1;FLOWDIRECTION_3 "Flow direction" true true true 2 Short 0 0,First,#,streams_from_featureservice,FLOWDIRECTION_3,-1,-1', 
                                None)

## Create starting points and run traces

#### Trout Production Streams

In [46]:
#Data sources
nj2015_swqs = os.path.join(home,"nhd_swqs_join") #original layer with just the swqs--fewest splits
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")
streams_TN = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network"
#arcpy.tn.EnableNetworkTopology(streams_TN)

In [53]:
#Dissolve lines and make starting points for traces
arcpy.management.Dissolve(nj2015_swqs, os.path.join(scratch1,"nj2015_diss_TP"), ['GNIS_NAME', 'TP'], "", "SINGLE_PART", "UNSPLIT_LINES")
in_features = os.path.join(scratch1,"nj2015_diss_TP")
arcpy.management.MakeFeatureLayer(in_features, 'TP_streams', "TP = 1")
arcpy.management.FeatureVerticesToPoints('TP_streams', os.path.join(scratch1,"TP_start_pts"), 'START') #Create starting points by selecting segments from each category (cdw, c1, tp, tm) and creating a start point. 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\TP_start_pts'>

In [54]:
#Trace all upstream for Trout Production Streams
arcpy.Trace_tn(in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network", 
               trace_type="UPSTREAM", 
               starting_points=os.path.join(scratch1,"TP_start_pts"), 
               barriers="", 
               result_types="AGGREGATED_GEOMETRY", 
               aggregated_lines = os.path.join(home, "TP_upstream_trace"))

<Result 'Trace_Results_Aggregated_Points'>

In [44]:
#Erase TP from TP upstream
in_features = os.path.join(scratch1,"nj2015_diss_TP")
arcpy.management.MakeFeatureLayer(in_features, 'TP_streams', "TP = 1")
arcpy.analysis.Erase(os.path.join(home, "TP_upstream_trace"), 'TP_streams', os.path.join(home, "TP_upstream_only"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\TP_upstream_only'>

In [49]:
#Select upstream segments 
arcpy.management.MakeFeatureLayer(nj2015_temp2, "lyr_streams")
arcpy.management.SelectLayerByLocation("lyr_streams", "SHARE_A_LINE_SEGMENT_WITH", os.path.join(home, "TP_upstream_only"))

<Result 'lyr_streams'>

In [50]:
#Add TP upstream column
input_fc = nj2015_temp2
field_name = "TP_upstream"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
    arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("lyr_streams",["OID@"])]
oidfield = arcpy.Describe("lyr_streams").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[1] = 0
        curs.updateRow(row)
del curs

Field exists


#### Category 1 Streams

##### Run main trace and clip operation

In [24]:
#Data sources
nj2015_swqs = os.path.join(home,"nhd_swqs_join") #original layer with just the swqs--fewest splits
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")
streams_TN = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network"
#arcpy.tn.EnableNetworkTopology(streams_TN)

In [6]:
#Dissolve lines and make starting points for traces
arcpy.management.MakeFeatureLayer(nj2015_temp2, "C1_streams", where_clause = "C1 = 1 And regulated = 1")
#arcpy.management.Dissolve('C1_streams', os.path.join(scratch1,"diss_C1"), ['GNIS_NAME', 'C1'], "", "SINGLE_PART", "UNSPLIT_LINES")
arcpy.management.FeatureVerticesToPoints("C1_streams", 
                                         os.path.join(scratch1,"C1_start_pts"), 
                                         'START') 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_start_pts'>

In [9]:
#Inner buffer before intersect to avoid the inaccuracies of the HUC14 layer
arcpy.analysis.Buffer(huc14s, 
                      os.path.join(scratch1, "huc14_00ft_inner_buffer"), 
                      buffer_distance_or_field= "-300 feet", 
                      dissolve_option = "NONE")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\huc14_00ft_inner_buffer'>

In [10]:
arcpy.analysis.Intersect(
    in_features=["C1_streams", os.path.join(scratch1, "huc14_00ft_inner_buffer")],
    out_feature_class=os.path.join(scratch1, "C1_streams_hucbuff_int"),
    join_attributes="ONLY_FID",
    cluster_tolerance=None,
    output_type="POINT"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_streams_hucbuff_int'>

In [11]:
#OR singlepoint
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "C1_streams_hucbuff_int"), os.path.join(scratch1, "C1_streams_hucbuff_SP"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_streams_hucbuff_SP'>

In [12]:
arcpy.management.Merge([os.path.join(scratch1,"C1_start_pts"), os.path.join(scratch1, "C1_streams_hucbuff_SP")], 
                       os.path.join(scratch1, "C1_starts_merge")
                      )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_starts_merge'>

In [13]:
arcpy.env.addOutputsToMap = True
#List HUC 14s with C1 in them
pt_layer = arcpy.MakeFeatureLayer_management(os.path.join(scratch1,"C1_start_pts"), "C1_start_pts_layer")
HUC14_map = arcpy.MakeFeatureLayer_management(r'X:\projects\njwrap\Data\NJDEP\Hydr_HUC14_bnd\Hydr_HUC14_bnd.gdb\Hydr_HUC14_bnd', "HUC14_map")
selection = arcpy.management.SelectLayerByLocation(HUC14_map, "CONTAINS", pt_layer, selection_type = "NEW_SELECTION")

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

HUC14s = unique_values(selection, 'HUC14')
arcpy.management.SelectLayerByAttribute(HUC14_map, "CLEAR_SELECTION")

<Result 'HUC14_map'>

In [18]:
#HUC_test = HUC14s[0]
len(HUC14s)
#HUC14s1 = HUC14s[311:]

392

In [14]:
#Test
 #
test_hucs = ['02020007020020', '02020007020030', '02020007010070', '02020007020060', '02040105040020']
#['02020007010030', '02020007010010', '02020007010020']

In [20]:
#Create empty feature class
hucclips = arcpy.management.CreateFeatureclass(home, 
                                               "C1_HUC14_1_28", 
                                               geometry_type = "POLYLINE", 
                                               template = os.path.join(scratch1, "stream_clippy_1_02020007010010"),  
                                               spatial_reference = arcpy.Describe(os.path.join(scratch1, "stream_clippy_1_02020007010010")).spatialReference 
                                              )

In [21]:
# runtime was 373.18712785243986 minutes
arcpy.env.addOutputsToMap = False
i = 1
hucs = HUC14s
list_length = len(hucs)
HUC14_map = arcpy.MakeFeatureLayer_management(r'X:\projects\njwrap\Data\NJDEP\Hydr_HUC14_bnd\Hydr_HUC14_bnd.gdb\Hydr_HUC14_bnd', "HUC14_map")
pt_layer = arcpy.MakeFeatureLayer_management(os.path.join(scratch1, "C1_starts_merge"), "C1_start_pts_layer")
agg_lines = os.path.join(scratch1, "temp_C1HUC_test3")
error_list = [] #list of hucs caused an error

start = time.time()
for huc in hucs:
    try:
        start1 = time.time()
        where = ("HUC14 = '" + huc +"'")
        select_huc = arcpy.management.MakeFeatureLayer(HUC14_map, "select_huc", where)
        selection = arcpy.management.SelectLayerByLocation(pt_layer, "WITHIN", select_huc, selection_type = "NEW_SELECTION")
        output_pts = arcpy.management.CopyFeatures(selection, os.path.join(scratch1, "huc_pts"))#must be features, does not honor selection or layer
        
        #print(f"C1 start points within {huc} selected")
        arcpy.Trace_tn(in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network", 
                       trace_type="UPSTREAM", 
                       starting_points= output_pts, 
                       barriers="", 
                       result_types="AGGREGATED_GEOMETRY",
                       clear_all_previous_trace_results = "CLEAR_ALL_PREVIOUS_TRACE_RESULTS",
                       aggregated_lines = agg_lines)
        #print("Upstream trace done")
        output_clip = arcpy.analysis.Clip(agg_lines, select_huc, os.path.join(scratch1, "temp_huc14_clips"))
        arcpy.management.Append(output_clip, hucclips) #Append clip to new fc
    except:
        error_list.append(huc)
    finally: 
        elapsed_time = time.time() - start1
        print(f"{i} of {list_length} complete in {elapsed_time} seconds")
        i = i+1
end = (time.time() - start)/360
error_list
print(f"total runtime was {end} hours")
arcpy.env.addOutputsToMap = True
error_list

1 of 392 complete in 25.037843465805054 seconds
2 of 392 complete in 31.036492347717285 seconds
3 of 392 complete in 31.324048042297363 seconds
4 of 392 complete in 29.10905933380127 seconds
5 of 392 complete in 30.15512442588806 seconds
6 of 392 complete in 29.070045709609985 seconds
7 of 392 complete in 28.947082996368408 seconds
8 of 392 complete in 29.189042329788208 seconds
9 of 392 complete in 47.4405882358551 seconds
10 of 392 complete in 55.40707230567932 seconds
11 of 392 complete in 54.36269736289978 seconds
12 of 392 complete in 54.95132827758789 seconds
13 of 392 complete in 58.45238018035889 seconds
14 of 392 complete in 57.252164125442505 seconds
15 of 392 complete in 59.45846652984619 seconds
16 of 392 complete in 58.74195408821106 seconds
17 of 392 complete in 59.74283218383789 seconds
18 of 392 complete in 54.92136549949646 seconds
19 of 392 complete in 54.70310091972351 seconds
20 of 392 complete in 54.64168691635132 seconds
21 of 392 complete in 54.33280873298645 sec

335 of 392 complete in 45.267879247665405 seconds
336 of 392 complete in 45.78307557106018 seconds
337 of 392 complete in 45.77750873565674 seconds
338 of 392 complete in 44.89330434799194 seconds
339 of 392 complete in 46.04755973815918 seconds
340 of 392 complete in 46.8677442073822 seconds
341 of 392 complete in 44.909703493118286 seconds
342 of 392 complete in 47.824540853500366 seconds
343 of 392 complete in 52.83918309211731 seconds
344 of 392 complete in 44.90011167526245 seconds
345 of 392 complete in 45.85874795913696 seconds
346 of 392 complete in 45.09551119804382 seconds
347 of 392 complete in 69.38450622558594 seconds
348 of 392 complete in 54.897093772888184 seconds
349 of 392 complete in 52.67980694770813 seconds
350 of 392 complete in 45.08375597000122 seconds
351 of 392 complete in 51.056915044784546 seconds
352 of 392 complete in 49.5820152759552 seconds
353 of 392 complete in 48.68724989891052 seconds
354 of 392 complete in 53.37075448036194 seconds
355 of 392 comple

[]

In [40]:
arcpy.env.addOutputsToMap = True
error_list

[]

##### Mopping up missed segments

In [25]:
c1 = arcpy.management.MakeFeatureLayer(nj2015_temp2, "C1_streams", where_clause = "C1 = 1 And regulated = 1")
#arcpy.analysis.Erase(hucclips, "C1_streams", os.path.join(home, "C1_upstream_huc_erase"))

In [24]:
c1_and_upstream = arcpy.management.Merge(["C1_streams", 
                                          os.path.join(home, "C1_upstream_huc_erase")], 
                                         os.path.join(scratch1, "C1_andupstream_merge"))

In [8]:
#Run upstream C1 from C1 endpoints
arcpy.management.FeatureVerticesToPoints("C1_streams", 
                                         os.path.join(scratch1,"C1_end_pts"), 
                                         'END') 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_end_pts'>

In [9]:
#Trace all upstream for Trout Production Streams
arcpy.Trace_tn(in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network", 
               trace_type="UPSTREAM", 
               starting_points=os.path.join(scratch1,"C1_end_pts"), 
               barriers="", 
               result_types="AGGREGATED_GEOMETRY", 
               aggregated_lines = os.path.join(home, "C1_upstream_trace_all"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\Trace_Results_Aggregated_Points'>

In [6]:
#Capture any stubs that were clipped that shouldn't have been
#After trace, erase trace from all upstream lines. 
arcpy.analysis.Erase(os.path.join(home, "C1_upstream_trace_all"), os.path.join(scratch1, "C1_andupstream_merge"), os.path.join(scratch1, "non_C1_upstream"))
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "non_C1_upstream"), os.path.join(scratch1, "non_C1_upstream_SP"))


#An alternative is to use segment ID or COMID. This could be good, but maybe many steps? Not sure how to implement
#Spatial join Hucs IDs to original stream network OID to C1 upstream. Then spatial join hucs by count if count > 1 select and assign to original HUC . 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\non_C1_upstream_SP'>

In [7]:
#Erase HUCs not used to cut down buffer time. 
# Path to the Layer
arcpy.management.MakeFeatureLayer(huc14s, "huc14_lyr")

# Integer list of OBJECTID(s)
test_hucs = ['02020007020020', '02020007020030', '02020007010070', '02020007020060', '02040105040020']
shucs = ", ".join(f"'{s}'" for s in test_hucs)
where_clause = f"HUC14 IN ({shucs})"

print(where_clause)
# Tool Execution
#arcpy.management.SelectLayerByAttribute(lyr, "NEW_SELECTION", where_clause)‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍


HUC14 in ('02020007020020', '02020007020030', '02020007010070', '02020007020060', '02040105040020')


In [7]:
field_name = "transfer_code"
target_lines = os.path.join(scratch1, "non_C1_upstream_SP")
if len(arcpy.ListFields(target_lines, field_name)) == 0:
    print("Field does not exist")       
else:
    arcpy.management.DeleteField(target_lines, field_name)

Field does not exist


In [8]:
buffer0 = arcpy.analysis.Buffer(target_lines, 
                                os.path.join(scratch1,"non_c1_buffer"), 
                                "1 Feet", 
                                "FULL", 
                                "ROUND", 
                                "ALL", 
                                None, 
                                "PLANAR")

In [9]:
buffer1 = arcpy.management.MultipartToSinglepart(buffer0, os.path.join(scratch1,"c1_upstream_buffer_MultipartToSi1"))

In [10]:
#Create composite non-c1 segments to select between a and upstream segments that are already short enough and don't need to be traced
#https://gis.stackexchange.com/questions/174752/dissolve-a-polyline-feature-class-so-that-touching-features-dissolve-into-a-sing
field_name = "transfer_code"
target_lines = os.path.join(scratch1, "non_C1_upstream_SP")

    
#buffer0 = arcpy.analysis.Buffer(target_lines, os.path.join(scratch1,"non_c1_buffer"), "1 Feet", "FULL", "ROUND", "ALL", None, "PLANAR")
#buffer1 = arcpy.management.MultipartToSinglepart(buffer0, os.path.join(scratch1,"c1_upstream_buffer_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, field_name, "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(target_lines, 
                                         buffer1, 
                                         os.path.join(scratch1,"non_c1_buffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL")
non_C1_diss = arcpy.management.Dissolve(buffer_join, 
                                        os.path.join(scratch1, "non_c1_buffer_Dissolve"), 
                                        field_name, 
                                        None, 
                                        "MULTI_PART", 
                                        "DISSOLVE_LINES", '')

In [16]:
arcpy.management.FeatureVerticesToPoints(os.path.join(scratch1, "C1_andupstream_merge"), 
                                         os.path.join(scratch1,"C1_andupstream_start_pts"), 
                                         'START') 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_andupstream_start_pts'>

In [17]:
c1_starts = arcpy.MakeFeatureLayer_management(os.path.join(scratch1,"C1andupstream_start_pts"), "c1_starts_lyr")
non_c1_lyr = arcpy.MakeFeatureLayer_management(os.path.join(scratch1, "non_c1_buffer_Dissolve"), "non_c1_disslyr")


In [21]:
#Select c1 upstream segments that might have been missed for flukey network reasons
#Select any lines that intersect upstream trace or C1 start points that are less than a certain length.

#Select non-cdw segments that intersect start point and are less than 5280'
arcpy.management.SelectLayerByAttribute(non_c1_lyr, "NEW_SELECTION", "Shape_Length < 4000", None)
arcpy.management.SelectLayerByLocation(non_c1_lyr, "INTERSECT", c1_starts, selection_type = "SUBSET_SELECTION")


<Result 'non_c1_disslyr'>

In [22]:
arcpy.management.CopyFeatures(non_c1_lyr, os.path.join(scratch1, "C1_upstream_tips"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_upstream_tips'>

In [26]:
arcpy.management.SelectLayerByAttribute(non_c1_lyr, "NEW_SELECTION", "Shape_Length < 4000", None)
arcpy.management.SelectLayerByLocation(non_c1_lyr, "INTERSECT", c1, selection_type = "SUBSET_SELECTION")
arcpy.management.SelectLayerByLocation(non_c1_lyr, "INTERSECT", os.path.join(scratch1,"C1_end_pts"), selection_type = "SUBSET_SELECTION", invert_spatial_relationship = "INVERT")

<Result 'non_c1_disslyr'>

In [27]:
arcpy.management.CopyFeatures(non_c1_lyr, os.path.join(scratch1, "C1_network_errors"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\C1_network_errors'>

In [28]:
arcpy.management.Merge([os.path.join(scratch1, "C1_upstream_tips"),
                                          os.path.join(scratch1, "C1_network_errors"),
                                          os.path.join(home, "C1_upstream_huc_erase")], 
                                         os.path.join(scratch1, "C1_upstream_final"))

arcpy.management.Dissolve(os.path.join(scratch1, "C1_upstream_final"), 
                          os.path.join(home, "C1_upstream_final"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\C1_upstream_final'>

In [30]:
arcpy.management.CopyFeatures(os.path.join(home, "C1_upstream_final"), os.path.join(background, "C1_upstream"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\C1_upstream'>

## Create trace network for TM/CDW--needs to be updated

### TM/CDW combined split network

In [11]:
#Set workspace and create variable for new copy of lines
nj2015_temp3 = "nhd_c1_split1"
streams_TN = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers1.gdb\Stream_buffers\TM_streams_network"

In [12]:
#Dissolve lines and make starting points for TM generic
arcpy.management.MakeFeatureLayer(nj2015_temp3, 'TM_streams', "TM = 1 And regulated = 1")
arcpy.management.Dissolve('TM_streams', "nhd_diss_TM", ['GNIS_NAME', 'TM'], "", "SINGLE_PART", "UNSPLIT_LINES")
in_features = "nhd_diss_TM"
arcpy.management.FeatureVerticesToPoints(in_features, "TM_start_pts", 'START') #Create starting points by selecting segments from each category (cdw, c1, tp, tm) and creating a start point. 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\scratch.gdb\\TM_start_pts'>

In [26]:
#Trace all upstream for Trout Maintenance Streams
arcpy.env.workspace = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015"
selected_streams = arcpy.Trace_tn(in_trace_network="streams_featureservice_Network", 
                                  trace_type="UPSTREAM", 
                                  starting_points="TM_start_pts", 
                                  barriers="", 
                                  result_types="AGGREGATED_GEOMETRY", 
                                  aggregated_lines = os.path.join(scratch, "TM_upstream_temp"))

In [27]:
# cdw upstream generic trace
#Dissolve lines and make starting points for cdw generic
arcpy.management.MakeFeatureLayer(nj2015_temp3, 'cdw_streams', "all_cdw = 1 And regulated = 1")
arcpy.management.Dissolve('cdw_streams', "nhd_diss_cdw", ['GNIS_NAME', 'all_cdw'], "", "SINGLE_PART", "UNSPLIT_LINES")
in_features = "nhd_diss_cdw"
arcpy.management.FeatureVerticesToPoints(in_features, "cdw_start_pts", 'START') #Create starting points by selecting segments from each category (cdw, c1, tp, tm) and creating a start point. 

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\Hydr_NHD_2015.gdb\\Hydr_NHD_2015\\cdw_start_pts'>

In [28]:
#Trace all upstream for CDW Streams
arcpy.env.workspace = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015"
selected_streams = arcpy.Trace_tn(in_trace_network="streams_featureservice_Network", 
                                  trace_type="UPSTREAM", 
                                  starting_points="cdw_start_pts", 
                                  barriers="", 
                                  result_types="AGGREGATED_GEOMETRY", 
                                  aggregated_lines = os.path.join(scratch, "cdw_upstream_temp"))

##### Network set up

In [ ]:
# select TM and cdw upstream from stream network
#generate points every 264'
# split network by point
# Create network and troubleshoot

In [ ]:
"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers.gdb\Stream_buffers\TM_all_upstream_264_split1"

In [ ]:
#SET UP TRACE NETWORK
#Point along line every 264, split by points
#Repair Geometry
arcpy.management.RepairGeometry(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers\TM_all_upstream_264_split1", "DELETE_NULL", "ESRI")

#Add attribute for shape length that isn't shape length that can be added to network
#arcpy.management.CalculateField(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers\TM_all_upstream_264_split1", "Segment_Length", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#Create trace network
arcpy.tn.CreateTraceNetwork(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers", "TM_streams_network", None, "TM_all_upstream_264_split1 SIMPLE_EDGE")

#Add network attribute
arcpy.tn.AddNetworkAttribute(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers\TM_streams_network", "Segment_Length", "DOUBLE", "NULLABLE")

#Set network attribute
arcpy.tn.SetNetworkAttribute(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers\TM_streams_network", "Segment_Length", "TM_all_upstream_264_split1", "Segment_Length")

#Enable network topology
#Crashes when run from python window. Run from Geoprocessing pane

#There were a number of duplicate vertices. Unfortunately I could not find a way to fix these automatically and had to manually delete vertices. When deleting some vertices, it changed the line so that the trace points were no longer on the line. Points should be snapped back to the lines before tracing. 

In [ ]:
arcpy.env.workspace = r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers"

#Select and copy CDW upstream lines. The ones from the trace are all dissolved so the generate points along lines tool fails
arcpy.analysis.Clip(in_features, "CDW_upstream", "CDW_upstream_clip")

#Generate points along lines every 264'
arcpy.management.GeneratePointsAlongLines("CDW_upstream_clip", r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\CDW_264_pts", "DISTANCE", "264 Feet", None, None)

#Split lines at 264' points 
arcpy.management.SplitLineAtPoint("CDW_upstream_clip", "CDW_264_pts", "CDW_264_split", "1 Feet")

In [ ]:
arcpy.env.workspace = r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers"

In [ ]:
#Create and troubleshoot TRACE NETWORK
#Repair Geometry
#arcpy.management.RepairGeometry("CDW_264_split", "DELETE_NULL", "ESRI")

#Add attribute for shape length that isn't shape length that can be added to network
arcpy.management.CalculateField("CDW_264_split", "Segment_Length", "!Shape_Length!", "PYTHON3", '', "DOUBLE", "NO_ENFORCE_DOMAINS")

#Create trace network
arcpy.tn.CreateTraceNetwork(r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers", "CDW_streams_network", None, "CDW_264_split SIMPLE_EDGE")

#Add network attribute
arcpy.tn.AddNetworkAttribute("CDW_streams_network", "Segment_Length", "DOUBLE", "NULLABLE")

#Set network attribute
arcpy.tn.SetNetworkAttribute("CDW_streams_network", "Segment_Length", "CDW_264_split", "Segment_Length")

#Enable network topology
arcpy.tn.EnableNetworkTopology("CDW_streams_network") #Crashes when run from python window. Run from Geoprocessing pane

#Check Line errors and edit as needed, see script below to delete duplicate vertices


In [ ]:
#Remove Duplicate Vertices from network if needed

import arcpy
import math
#https://tereshenkov.wordpress.com/2017/11/08/removing-redundant-polyline-vertices-using-arcpy/
#https://gis.stackexchange.com/questions/226982/delete-middle-vertex-of-multiple-3-vertex-polylines

#Remove Duplicate Vertices

#———————————————————————-
def distance_between(p1, p2, p3, tolerance=0.1):
    """return True if 3 points are collinear.
    tolerance value will decide whether lines are collinear; may need
    to adjust it based on the XY tolerance value used for feature class"""
    x1, y1 = p1[0], p1[1]
    x2, y2 = p2[0], p2[1]
    x3, y3 = p3[0], p3[1]
    res1 = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    res2 = math.sqrt((x3 - x2)**2 + (y3 - y2)**2)
    if (-tolerance <= res1 <= tolerance) or (-tolerance <= res2 <= tolerance):
        return True


#———————————————————————-
def get_redundant_vertices(vertices):
    """get redundant vertices from a line shape vertices"""
    indexes_of_vertices_to_remove = []
    start_idx, middle_index, end_index = 0, 1, 2
    for i in range(len(vertices)):
        p1, p2, p3 = vertices[start_idx:end_index + 1]
        if distance_between(p1, p2, p3):
            indexes_of_vertices_to_remove.append(middle_index)

        start_idx += 1
        middle_index += 1
        end_index += 1
        if end_index == len(vertices):
            break
    return indexes_of_vertices_to_remove


#———————————————————————-
def clean_geometries(fc, densify_curves=False):
    """clean polyline features in the fc removing redundant vertices"""
    in_sr = arcpy.Describe(fc).spatialReference.factoryCode

    with arcpy.da.UpdateCursor(fc, ['SHAPE@', 'OID@']) as ucur:
        for row in ucur:
            #print("OBJECTID", row[1])
            cleaned_parts = []
            shape = row[0]

            for part in range(shape.partCount):
                vertices = [(p.X, p.Y) for p in shape.getPart(part)]
                if len(vertices) < 3:  #polyline's part consists of 2 vertices
                    continue
                vertices_to_remove = get_redundant_vertices(vertices)
                vertices_to_keep = [
                    val for idx, val in enumerate(vertices)
                    if idx not in vertices_to_remove
                ]
                cleaned_part_as_array = arcpy.Array(
                    [arcpy.Point(*coords) for coords in vertices_to_keep])
                cleaned_parts.append(cleaned_part_as_array)

            if cleaned_parts:
                cleaned_shape = arcpy.Polyline(
                    arcpy.Array(cleaned_parts), in_sr)
                row[0] = cleaned_shape
                ucur.updateRow(row)



#fc = "CDW_264_split"  
#clean_geometries(fc, densify_curves=False)
workspace = r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb"
with arcpy.da.Editor(workspace) as edit:
    fc = r"C:\Users\kdd56\Documents\Kate_GIS_Local\Stream_buffers\stream_buffers.gdb\Stream_buffers\CDW_264_split"  
    clean_geometries(fc, densify_curves=False)

## Create starting points and run traces

### Trout Maintenance

#### TM Trace Point selection

In [8]:
#Set workspace and create variable for new copy of lines
nj2015_swqs = os.path.join(home,"nhd_swqs_join")
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")

# See TM/CDW combined split network to run a generic upstream trace to help cut down the lines in the network

In [9]:
arcpy.env.addOutputsToMap = True
arcpy.management.MakeFeatureLayer(nj2015_swqs, 'TM_streams_lyr', "TM = 1")
non_TM = arcpy.management.MakeFeatureLayer(nj2015_temp2, "non-TM_lyr", "TM <> 1")

In [46]:
arcpy.management.FeatureVerticesToPoints('TM_streams_lyr', os.path.join(scratch1, "TM_end_pts"), 'END')
arcpy.analysis.Buffer(os.path.join(scratch1, "TM_end_pts"), 
                      os.path.join(scratch1, "TM_end_pt_buff"), 
                      buffer_distance_or_field = '10 feet'
                     )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\TM_clip_end_pts'>

In [47]:
arcpy.analysis.Erase('TM_streams_lyr', 
                    os.path.join(scratch1, "TM_end_pt_buff"), 
                   os.path.join(scratch1, "TM_stream_buff_clips"))
arcpy.management.FeatureVerticesToPoints(os.path.join(scratch1, "TM_stream_buff_clips"), os.path.join(scratch1, "TM_clip_end_pts"), 'END')

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\TM_clip_end_pts'>

In [50]:
arcpy.management.FeatureVerticesToPoints('TM_streams_lyr', os.path.join(scratch1, "TM_start_pts"), 'START')
arcpy.management.Merge([os.path.join(scratch1, "TM_start_pts"), os.path.join(scratch1, "TM_clip_end_pts")], os.path.join(scratch1, "TM_mod_pts"))
untraced_waterways = arcpy.management.MakeFeatureLayer(nj2015_polys, "Delaware_and_canals", where_clause = "GNIS_NAME = 'Delaware River' Or GNIS_NAME = 'Delaware and Raritan Canal'")

mod_pts = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "TM_mod_pts"), "TM_mod_pts_lyr")
arcpy.management.SelectLayerByLocation(mod_pts, 
                                       "INTERSECT", 
                                       untraced_waterways, 
                                       selection_type = "NEW_SELECTION", invert_spatial_relationship = "INVERT")

<Result 'TM_mod_pts_lyr'>

In [56]:
if arcpy.Exists(os.path.join(scratch1, "selected_TM_mods")):
    print("File is here. Let me clean that up for you")
    arcpy.management.Delete(os.path.join(scratch1, "selected_TM_mods")) 
else: 
    print("Feature class doesn't exist")

arcpy.management.CopyFeatures(mod_pts, os.path.join(scratch1, "selected_TM_mods"))
arcpy.edit.Snap(os.path.join(scratch1, "selected_TM_mods"), 
                [[r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_from_featureservice", 'EDGE', '3 feet']])

File is here. Let me clean that up for you


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\selected_TM_mods'>

In [57]:
#Get all upstream reaches
streams_TN = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network"
starting_pts = os.path.join(scratch1, "selected_TM_mods")
agglines = os.path.join(lscratch, "TM_upstream_all")

if arcpy.Exists(agglines):
    print("File is here. Let me clean that up for you")
    arcpy.management.Delete(agglines) 
else: 
    print("Feature class doesn't exist")
    
selected_streams = arcpy.Trace_tn(in_trace_network=streams_TN, 
                                  trace_type="UPSTREAM", 
                                  starting_points= starting_pts, 
                                  barriers="", 
                                  result_types="AGGREGATED_GEOMETRY", 
                                  aggregated_lines = agglines
                                 )

Feature class doesn't exist


In [58]:
#Erase actual TM streams from upstream
agglines = os.path.join(lscratch, "TM_upstream_all")
arcpy.management.MultipartToSinglepart(agglines,os.path.join(scratch1, "TM_upstream_all_SP"))
arcpy.analysis.Erase(os.path.join(scratch1, "TM_upstream_all_SP"), 'TM_streams_lyr', os.path.join(scratch1, "TM_upstream_erase"))


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\TM_upstream_all_SP'>

In [24]:
arcpy.management.MakeFeatureLayer(nj2015_swqs, 'TM_streams_lyr', "TM = 1")

<Result 'TM_streams_lyr'>

In [39]:
#Create composite non-TM segments to select between a and upstream segments that are already short enough and don't need to be traced
#https://gis.stackexchange.com/questions/174752/dissolve-a-polyline-feature-class-so-that-touching-features-dissolve-into-a-sing
buffer = arcpy.analysis.Buffer(os.path.join(scratch1, "TM_upstream_erase"), os.path.join(scratch1,"non_TM_buffer"), "1 Feet", "FULL", "ROUND", "ALL", None, "PLANAR")
buffer1 = arcpy.management.MultipartToSinglepart(buffer, os.path.join(scratch1,"TM_upstream_buffer_MultipartToSi1"))

field_name = "transfer_code"
target_lines = os.path.join(scratch1, "TM_upstream_erase")
if len(arcpy.ListFields(target_lines, field_name)) == 0:
    print("Field does not exists")
         
else:
    arcpy.management.DeleteField(target_lines, field_name)
        
arcpy.management.CalculateField(buffer1, field_name, "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(target_lines, buffer1, os.path.join(scratch1,"non_TM_buffer_join"), "JOIN_ONE_TO_ONE", "KEEP_ALL")
non_TM_diss = arcpy.management.Dissolve(buffer_join, os.path.join(scratch1, "non_TM_buffer_Dissolve"), field_name, None, "MULTI_PART", "DISSOLVE_LINES", '')

Field does not exists


In [50]:
#Upstream startpoints of TM segments
TM_starts = arcpy.MakeFeatureLayer_management(os.path.join(scratch1, "TM_start_pts"), "TM_starts_lyr")
non_TM_lyr = arcpy.MakeFeatureLayer_management(non_TM_diss, "non_TM_disslyr")

#Select non-TM segments that intersect start point and are less than 5280'
arcpy.management.SelectLayerByAttribute(non_TM_lyr, "NEW_SELECTION", "Shape_Length < 5280", None)
arcpy.management.SelectLayerByLocation(non_TM_lyr, "INTERSECT", TM_starts, selection_type = "SUBSET_SELECTION")

<Result 'non_TM_disslyr'>

In [51]:
if arcpy.Exists(os.path.join(home,"TM_Trace_results_partbeween_upstream")):
    arcpy.management.Delete(os.path.join(home,"TM_Trace_results_partbeween_upstream")) 
else: 
    print("Feature class doesn't exist")
bw_upstream = arcpy.management.CopyFeatures(non_TM_lyr, os.path.join(home,"TM_Trace_results_partbeween_upstream1a"))

In [ ]:
#These are TM upstream segments that can be added already. Any of the trace points that overlap the end points can be removed from the set
os.path.join(home,"TM_Trace_results_partbeween_upstream1a")

In [29]:
#arcpy.analysis.Intersect(['TM_streams_lyr', "non-TM_lyr"], os.path.join(scratch1, "tm_non_tm_intersections"), output_type = "POINT")
#all_intersections = arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "tm_non_tm_intersections"), os.path.join(scratch1, "tm_non_tm_int_SP"))
#arcpy.edit.Snap(all_intersections, [[r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_from_featureservice", 'EDGE', '3 feet']])

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\tm_non_tm_int_SP'>

In [35]:
#Select start points that DO NOT intersect the between/upstream lines already seleced
bw_upstream = os.path.join(home,"TM_Trace_results_partbeween_upstream1a")
TM_int = arcpy.MakeFeatureLayer_management(all_intersections, "TM_starts_lyr")
arcpy.management.SelectLayerByLocation(TM_int, "INTERSECT", bw_upstream, invert_spatial_relationship = "INVERT")

<Result 'TM_starts_lyr'>

In [36]:
if arcpy.Exists(os.path.join(scratch1, "selected_TM_trace_pts")):
    print("file is here. Let me get rid of that for you.")
    arcpy.management.Delete(os.path.join(scratch1, "selected_TM_trace_pts")) 
    arcpy.management.CopyFeatures(TM_int, 
                              os.path.join(scratch1, "selected_TM_trace_pts"))
else: 
    print("Feature class doesn't exist")
    arcpy.management.CopyFeatures(TM_int, 
                              os.path.join(scratch1, "selected_TM_trace_pts"))

file is here. Let me get rid of that for you.


In [39]:
#Snap points to edited stream lines
arcpy.edit.Snap(os.path.join(scratch1, "selected_TM_trace_pts"), 
                r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers_1.gdb\Stream_buffers\CDW_264_split EDGE '5 Feet'")


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\selected_TM_trace_pts'>

#### Trace Loop

In [40]:
#List Starting Points
TM_trace_pts1 = os.path.join(scratch1,"selected_TM_trace_pts")
oid_fieldname = arcpy.Describe(TM_trace_pts1).OIDFieldName

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

TM_trace_oids = unique_values(TM_trace_pts1, oid_fieldname)


In [41]:
len(TM_trace_oids)

857

In [42]:
#Subset trace points if not tracing all in one loop
#TM_trace_oids = unique_values(TM_trace_pts1, oid_fieldname)

test_oids = [1, 3, 4, 5, 6, 8, 9, 10, 11]


In [44]:
#Inputs
TM_trace_pts1 = os.path.join(scratch1,"selected_TM_trace_pts")
oid_list = TM_trace_oids
agglines = os.path.join(lscratch, "TM_trace_results_part1")


#Set up
if arcpy.Exists(agglines):
    print("file is here. I will destroy it for you")
    arcpy.management.Delete(agglines) 
else: 
    print("Feature class doesn't exist")
#select upstream 1 mile
arcpy.env.addOutputsToMap = True
oid_fieldname = arcpy.Describe(TM_trace_pts1).OIDFieldName
i = 1
list_length = len(oid_list)
error_list_TM = [] #list of point oid that caused an error

t = time.localtime()
start_time = time.strftime("%H:%M:%S", t)
t1 = time.time()
print(f"start time is {start_time}")
      
for pt in oid_list:
    try:
        where = f"{oid_fieldname} = {pt}"
        start_pt = arcpy.conversion.ExportFeatures(TM_trace_pts1, os.path.join(lscratch,"temp_point"), where) 
        arcpy.tn.Trace(
            in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers_1.gdb\Stream_buffers\CDW_streams_network",
            trace_type="UPSTREAM",
            starting_points= start_pt,
            barriers="TN_Temp_Barriers",
            path_direction="NO_DIRECTION",
            shortest_path_network_attribute_name="",
            include_barriers="INCLUDE_BARRIERS",
            validate_consistency="VALIDATE_CONSISTENCY",
            ignore_barriers_at_starting_points="DO_NOT_IGNORE_BARRIERS_AT_STARTING_POINTS",
            allow_indeterminate_flow="IGNORE_INDETERMINATE_FLOW",
            condition_barriers=None,
            function_barriers="ADD Segment_Length IS_GREATER_THAN_OR_EQUAL_TO 5280 true",
            traversability_scope="BOTH_JUNCTIONS_AND_EDGES",
            functions=None,
            output_conditions=None,
            result_types="AGGREGATED_GEOMETRY",
            selection_type="NEW_SELECTION",
            clear_all_previous_trace_results="DO_NOT_CLEAR_ALL_PREVIOUS_TRACE_RESULTS",
            trace_name="",
            aggregated_points=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\RiparianZones_20240109.gdb\Trace_Results_Aggregated_Points",
            aggregated_lines= agglines, 
            out_network_layer=None
        )#The output geometry from the current trace operation will be appended to the feature classes storing aggregated geometry.
    except:
        print(f"{pt} failed")
        error_list_TM.append(pt)
    finally:
        print(f"{i} of {list_length} complete")
        i = i+1
print(error_list_TM)

t = time.localtime()
end_time = time.strftime("%H:%M:%S", t)
print(end_time)
t2 = time.time()
elapsed_time = ((t2 - t1)/60)
print(f"loop took {elapsed_time} minutes to run")

file is here. I will destroy it for you
start time is 14:04:10
1 of 857 complete
2 of 857 complete
3 of 857 complete
4 of 857 complete
5 of 857 complete
6 of 857 complete
7 of 857 complete
8 of 857 complete
9 of 857 complete
10 of 857 complete
11 of 857 complete
12 of 857 complete
13 of 857 complete
14 of 857 complete
15 of 857 complete
16 of 857 complete
17 of 857 complete
18 of 857 complete
19 of 857 complete
20 of 857 complete
21 of 857 complete
22 of 857 complete
23 of 857 complete
24 of 857 complete
25 of 857 complete
26 of 857 complete
27 of 857 complete
28 of 857 complete
29 of 857 complete
30 of 857 complete
31 of 857 complete
32 of 857 complete
33 of 857 complete
34 of 857 complete
35 failed
35 of 857 complete
36 of 857 complete
37 of 857 complete
38 of 857 complete
39 of 857 complete
40 of 857 complete
41 of 857 complete
42 of 857 complete
43 of 857 complete
44 of 857 complete
45 of 857 complete
46 of 857 complete
47 of 857 complete
48 of 857 complete
49 of 857 complete
50 of

785 of 857 complete
786 of 857 complete
787 of 857 complete
788 of 857 complete
789 of 857 complete
790 of 857 complete
791 of 857 complete
792 of 857 complete
793 of 857 complete
794 of 857 complete
795 of 857 complete
796 of 857 complete
797 of 857 complete
798 of 857 complete
799 of 857 complete
800 of 857 complete
801 of 857 complete
802 of 857 complete
803 of 857 complete
804 of 857 complete
805 of 857 complete
806 of 857 complete
807 of 857 complete
808 of 857 complete
809 of 857 complete
810 of 857 complete
811 of 857 complete
812 of 857 complete
813 of 857 complete
814 of 857 complete
815 of 857 complete
816 of 857 complete
817 of 857 complete
818 of 857 complete
819 of 857 complete
820 of 857 complete
821 of 857 complete
822 of 857 complete
823 of 857 complete
824 of 857 complete
825 of 857 complete
826 of 857 complete
827 of 857 complete
828 of 857 complete
829 of 857 complete
830 of 857 complete
831 of 857 complete
832 of 857 complete
833 of 857 complete
834 of 857 complete


In [ ]:
#Error list

[35, 56, 187, 188, 190, 191, 192, 193, 194, 196, 197, 198, 199, 200, 203, 204, 205, 207, 208, 211, 212, 213, 214, 215, 216, 248, 249, 312, 313, 314, 315, 356, 357, 394, 395, 396, 397, 449, 450, 518, 519, 520, 552, 581, 647, 648, 649, 716, 717, 722, 723, 734, 735, 744, 745, 754, 755, 766, 767, 778, 779, 780, 781, 782, 783, 784, 785, 837, 839, 840, 841, 842, 845, 846, 849, 850, 851]

In [47]:
#Get just the upstream lines with the TM removed
agglines = os.path.join(lscratch, "TM_trace_results_part1")
arcpy.analysis.Erase(agglines, "TM_streams_lyr", os.path.join(home, "TM_trace_results_erase"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\TM_trace_results_erase'>

In [48]:
#Find any untraced points
TM_trace_pts1 = os.path.join(scratch1, "selected_TM_trace_pts")

arcpy.management.MakeFeatureLayer(TM_trace_pts1, "TM_trace_lyr")
arcpy.management.SelectLayerByLocation("TM_trace_lyr", "INTERSECT", os.path.join(home, "TM_trace_results_erase"), invert_spatial_relationship = "INVERT")

<Result 'TM_trace_lyr'>

In [4]:
final_TM_pts = arcpy.management.CopyFeatures("TM_trace_lyr", os.path.join(scratch1, "TM_temp"))
#arcpy.edit.Snap(final_TM_pts, [[r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_from_featureservice", 'END', '5 feet']])

In [5]:
#List Starting Points
oid_fieldname = arcpy.Describe(final_TM_pts).OIDFieldName

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

TM_trace_oids1 = unique_values(final_TM_pts, oid_fieldname)


In [6]:
len(TM_trace_oids1)

192

In [7]:
#Inputs
final_TM_pts = os.path.join(scratch1, "TM_temp")
oid_list = TM_trace_oids1
agglines = os.path.join(lscratch, "TM_trace_results_part2")


#Set up
if arcpy.Exists(agglines):
    print("file is here. I will destroy it for you")
    arcpy.management.Delete(agglines) 
else: 
    print("Feature class doesn't exist")
#select upstream 1 mile
arcpy.env.addOutputsToMap = True

oid_fieldname = arcpy.Describe(final_TM_pts).OIDFieldName
i = 1
list_length = len(oid_list)
error_list_TM = [] #list of point oid that caused an error
t = time.localtime()
start_time = time.strftime("%H:%M:%S", t)
t1 = time.time()
print(f"start time is {start_time}")

for pt in oid_list:
    try:
        where = f"{oid_fieldname} = {pt}"
        start_pt = arcpy.conversion.ExportFeatures(final_TM_pts, os.path.join(scratch1,"temp_point"), where) 
        #print("Upstream trace done")
        arcpy.tn.Trace(
            in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network",
            trace_type="UPSTREAM",
            starting_points=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\scratch1.gdb\temp_point",
            barriers="TN_Temp_Barriers",
            path_direction="NO_DIRECTION",
            shortest_path_network_attribute_name="",
            include_barriers="INCLUDE_BARRIERS",
            validate_consistency="VALIDATE_CONSISTENCY",
            ignore_barriers_at_starting_points="DO_NOT_IGNORE_BARRIERS_AT_STARTING_POINTS",
            allow_indeterminate_flow="IGNORE_INDETERMINATE_FLOW",
            condition_barriers=None,
            function_barriers="ADD 'Shape length' IS_GREATER_THAN_OR_EQUAL_TO 5280 true",
            traversability_scope="BOTH_JUNCTIONS_AND_EDGES",
            functions=None,
            output_conditions=None,
            result_types="AGGREGATED_GEOMETRY",
            selection_type="NEW_SELECTION",
            clear_all_previous_trace_results="DO_NOT_CLEAR_ALL_PREVIOUS_TRACE_RESULTS",
            trace_name="",
            aggregated_points=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\RiparianZones_20240109.gdb\Trace_Results_Aggregated_Points",
            aggregated_lines= agglines, #os.path.join(lscratch,"TM_trace_part2"),
            out_network_layer=None)#The output geometry from the current trace operation will be appended to the feature classes storing aggregated geometry.
    except:
        print(f"{pt} failed")
        error_list_TM.append(pt)
    finally:
        print(f"{i} of {list_length} complete")
        i = i+1
print(error_list_TM)

t = time.localtime()
end_time = time.strftime("%H:%M:%S", t)
print(end_time)
t2 = time.time()
elapsed_time = ((t2 - t1)/60)
print(f"loop took {elapsed_time} minutes to run")

Feature class doesn't exist
start time is 22:52:49
1 of 192 complete
2 of 192 complete
3 of 192 complete
4 of 192 complete
5 of 192 complete
6 of 192 complete
7 of 192 complete
8 failed
8 of 192 complete
9 of 192 complete
10 of 192 complete
11 of 192 complete
12 of 192 complete
13 of 192 complete
14 of 192 complete
15 of 192 complete
16 of 192 complete
17 of 192 complete
18 of 192 complete
19 of 192 complete
20 of 192 complete
21 of 192 complete
22 of 192 complete
23 of 192 complete
24 of 192 complete
25 of 192 complete
26 of 192 complete
27 of 192 complete
28 of 192 complete
29 of 192 complete
30 of 192 complete
31 of 192 complete
32 of 192 complete
33 of 192 complete
34 of 192 complete
35 of 192 complete
36 of 192 complete
37 of 192 complete
38 of 192 complete
39 of 192 complete
40 of 192 complete
41 of 192 complete
42 of 192 complete
43 of 192 complete
44 of 192 complete
45 of 192 complete
46 of 192 complete
47 of 192 complete
48 of 192 complete
49 of 192 complete
50 of 192 complete

In [34]:
#Merge all results files
#arcpy.management.Merge([os.path.join(home, "TM_Trace_results_partbeween_upstream1a"), os.path.join(lscratch, "TM_trace_results_part1"), os.path.join(lscratch,"TM_trace_part2")], os.path.join(home, "TM_Trace_results_all_merged"))
arcpy.management.Dissolve(os.path.join(home, "TM_Trace_results_all_merged"), os.path.join(home,"TM_Trace_results_all_dissolved"), multi_part = "SINGLE_PART")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\TM_Trace_results_all_dissolved'>

In [62]:
arcpy.analysis.Erase(os.path.join(home,"TM_Trace_results_all_dissolved"), 'TM_streams_lyr', os.path.join(home, "TM_trace_results_erase"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\TM_trace_results_erase'>

In [63]:
#Erase TM lines from upstream traces
arcpy.management.MakeFeatureLayer(os.path.join(home, "TM_trace_results_erase"), "TM_1mile_upstream")
arcpy.management.SelectLayerByLocation("TM_1mile_upstream", "SHARE_A_LINE_SEGMENT_WITH", os.path.join(scratch1, "TM_upstream_all_SP"))

<Result 'TM_1mile_upstream'>

In [64]:
arcpy.management.CopyFeatures("TM_1mile_upstream", os.path.join(home, "TM_1mile_upstream_final"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\TM_1mile_upstream_final'>

In [31]:
arcpy.management.CopyFeatures(os.path.join(home, "TM_1mile_upstream_final"), os.path.join(background, "TM_upstream"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\TM_upstream'>

### CDW--Species Critically Dependent on the Watercourse

#### CDW Trace Point selection

In [4]:
#Set workspace and create variable for new copy of lines
nj2015_swqs = os.path.join(home,"nhd_swqs_join")
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")

In [8]:
arcpy.env.addOutputsToMap = True
arcpy.management.MakeFeatureLayer(nj2015_temp2, 'cdw_streams_lyr', where_clause = "all_cdw = 1 And regulated = 1")
non_cdw = arcpy.management.MakeFeatureLayer(nj2015_temp2, "non-cdw_lyr", where_clause = "all_cdw <> 1 Or regulated <> 1")

In [12]:
arcpy.management.FeatureVerticesToPoints('cdw_streams_lyr', os.path.join(scratch1, "cdw_end_pts"), 'END')
arcpy.analysis.Buffer(os.path.join(scratch1, "cdw_end_pts"), 
                      os.path.join(scratch1, "cdw_end_pt_buff"), 
                      buffer_distance_or_field = '10 feet'
                     )
arcpy.analysis.Erase('cdw_streams_lyr', 
                    os.path.join(scratch1, "cdw_end_pt_buff"), 
                   os.path.join(scratch1, "cdw_stream_buff_clips"))
arcpy.management.FeatureVerticesToPoints(os.path.join(scratch1, "cdw_stream_buff_clips"), os.path.join(scratch1, "cdw_clip_end_pts"), 'END')

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_clip_end_pts'>

In [11]:
arcpy.management.FeatureVerticesToPoints('cdw_streams_lyr', os.path.join(scratch1, "cdw_start_pts"), 'START')
arcpy.management.Merge([os.path.join(scratch1, "cdw_start_pts"), os.path.join(scratch1, "cdw_clip_end_pts")], os.path.join(scratch1, "cdw_mod_pts"))


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_mod_pts'>

In [ ]:
if arcpy.Exists(os.path.join(scratch1, "selected_cdw_mods")):
    print("File is here. Let me clean that up for you")
    arcpy.management.Delete(os.path.join(scratch1, "selected_cdw_mods")) 
else: 
    print("Feature class doesn't exist")

arcpy.management.CopyFeatures(mod_pts, os.path.join(scratch1, "selected_cdw_mods"))

In [13]:
arcpy.edit.Snap(os.path.join(scratch1, "cdw_mod_pts"), 
                [[r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_from_featureservice", 'EDGE', '5 feet']])

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_mod_pts'>

In [14]:
#Get all upstream reaches
streams_TN = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_featureservice_Network"
starting_pts = os.path.join(scratch1, "cdw_mod_pts")
agglines = os.path.join(lscratch, "cdw_upstream_all")

if arcpy.Exists(agglines):
    print("File is here. Let me clean that up for you")
    arcpy.management.Delete(agglines) 
else: 
    print("Feature class doesn't exist")
    
selected_streams = arcpy.Trace_tn(in_trace_network=streams_TN, 
                                  trace_type="UPSTREAM", 
                                  starting_points= starting_pts, 
                                  barriers="", 
                                  result_types="AGGREGATED_GEOMETRY", 
                                  aggregated_lines = agglines
                                 )

Feature class doesn't exist


In [15]:
#Erase actual cdw streams from upstream
agglines = os.path.join(lscratch, "cdw_upstream_all")
arcpy.management.MultipartToSinglepart(agglines,os.path.join(scratch1, "cdw_upstream_all_SP"))
arcpy.analysis.Erase(os.path.join(scratch1, "cdw_upstream_all_SP"), 'cdw_streams_lyr', os.path.join(scratch1, "cdw_upstream_erase"))
arcpy.management.RepairGeometry(os.path.join(scratch1, "cdw_upstream_erase"))


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_upstream_erase'>

In [33]:
#Create composite non-cdw segments to select between a and upstream segments that are already short enough and don't need to be traced
#https://gis.stackexchange.com/questions/174752/dissolve-a-polyline-feature-class-so-that-touching-features-dissolve-into-a-sing

arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "cdw_upstream_erase"), os.path.join(scratch1, "cdw_upstream_erase_SP"))

field_name = "transfer_code"
target_lines = os.path.join(scratch1, "cdw_upstream_erase_SP")
if len(arcpy.ListFields(target_lines, field_name)) == 0:
    print("Field does not exist")       
else:
    arcpy.management.DeleteField(target_lines, field_name)
    
buffer = arcpy.analysis.Buffer(target_lines, os.path.join(scratch1,"non_cdw_buffer"), "1 Feet", "FULL", "ROUND", "ALL", None, "PLANAR")
buffer1 = arcpy.management.MultipartToSinglepart(os.path.join(scratch1,"non_cdw_buffer"), os.path.join(scratch1,"cdw_upstream_buffer_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, field_name, "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(target_lines, 
                                         buffer1, 
                                         os.path.join(scratch1,"non_cdw_buffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL")
non_cdw_diss = arcpy.management.Dissolve(buffer_join, 
                                        os.path.join(scratch1, "non_cdw_buffer_diss"), 
                                        field_name, 
                                        None, 
                                        "MULTI_PART", 
                                        "DISSOLVE_LINES", '')

Field does not exist


In [8]:
#Select cdw upstream segments that already meet the criteria
#cdw_starts = arcpy.MakeFeatureLayer_management(os.path.join(scratch1, "cdw_start_pts"), "cdw_starts_lyr")
non_cdw_lyr = arcpy.MakeFeatureLayer_management(os.path.join(scratch1,"non_cdw_buffer_diss"), "non_cdw_disslyr")

#Select non-cdw segments that intersect cdw but not the end point and are less than 5280'
arcpy.management.SelectLayerByAttribute(non_cdw_lyr, "NEW_SELECTION", "Shape_Length < 5280", None)
#arcpy.management.SelectLayerByLocation(non_cdw_lyr, "INTERSECT", cdw_starts, selection_type = "SUBSET_SELECTION")
arcpy.management.SelectLayerByLocation(non_cdw_lyr, "INTERSECT", "cdw_streams_lyr", selection_type = "SUBSET_SELECTION")
arcpy.management.SelectLayerByLocation(non_cdw_lyr, "INTERSECT", os.path.join(scratch1, "cdw_end_pts"), selection_type = "SUBSET_SELECTION", invert_spatial_relationship = "INVERT")

<Result 'non_cdw_disslyr'>

In [36]:
if arcpy.Exists(os.path.join(home,"cdw_Trace_results_partbeween_upstream")):
    arcpy.management.Delete(os.path.join(home,"cdw_Trace_results_partbeween_upstream")) 
else: 
    print("Feature class doesn't exist")
bw_upstream = arcpy.management.CopyFeatures(non_cdw_lyr, os.path.join(home,"cdw_Trace_results_partbeween_upstream"))

Feature class doesn't exist


In [ ]:
#These are cdw upstream segments that can be added already. Any of the trace points that overlap the end points can be removed from the set
os.path.join(home,"cdw_Trace_results_partbeween_upstream")

In [38]:
#Get the set of points at the intersection between cdw and non-cdw
arcpy.analysis.Intersect(['cdw_streams_lyr', non_cdw], os.path.join(scratch1, "cdw_non_cdw_intersections"), output_type = "POINT")
all_intersections = arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "cdw_non_cdw_intersections"), os.path.join(scratch1, "cdw_non_cdw_int_SP"))
arcpy.edit.Snap(all_intersections, [[r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\streams_from_featureservice", 'EDGE', '5 feet']])

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_non_cdw_int_SP'>

In [39]:
#Select start points that DO NOT intersect the between/upstream lines already seleced
bw_upstream = os.path.join(home,"cdw_Trace_results_partbeween_upstream")
cdw_int = arcpy.MakeFeatureLayer_management(all_intersections, "cdw_int_pts_lyr")
arcpy.management.SelectLayerByLocation(cdw_int, "INTERSECT", bw_upstream, invert_spatial_relationship = "INVERT")
arcpy.management.SelectLayerByLocation(cdw_int, "INTERSECT", 
                                       os.path.join(scratch1, "cdw_end_pts"), 
                                       selection_type = "SUBSET_SELECTION", 
                                       invert_spatial_relationship = "INVERT")

<Result 'cdw_int_pts_lyr'>

In [41]:
if arcpy.Exists(os.path.join(scratch1, "selected_cdw_trace_pts")):
    print("file is here. Let me get rid of that for you.")
    arcpy.management.Delete(os.path.join(scratch1, "selected_cdw_trace_pts")) 
    arcpy.management.CopyFeatures(cdw_int, 
                              os.path.join(scratch1, "selected_cdw_trace_pts"))
else: 
    print("Feature class doesn't exist")
    arcpy.management.CopyFeatures(cdw_int, 
                              os.path.join(scratch1, "selected_cdw_trace_pts"))

Feature class doesn't exist


In [42]:
#Snap points to edited stream lines
arcpy.edit.Snap(os.path.join(scratch1, "selected_cdw_trace_pts"), 
                r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers_1.gdb\Stream_buffers\CDW_264_split EDGE '5 Feet'")


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\selected_cdw_trace_pts'>

#### Network trace

In [3]:
#Get list of CDWstart points to trace
cdw_trace_pts1 = os.path.join(scratch1, "selected_cdw_trace_pts")
oid_fieldname = arcpy.Describe(cdw_trace_pts1).OIDFieldName

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

cdw_start_oids = unique_values(cdw_trace_pts1, oid_fieldname)

In [4]:
len(cdw_start_oids)

1419

In [45]:
#Subset trace points if not tracing all in one loop
#TM_trace_oids = unique_values(TM_trace_pts1, oid_fieldname)

test_oids = [1, 3, 4, 5, 6, 8, 9, 10, 11]


In [5]:
#Inputs
cdw_trace_pts1 = os.path.join(scratch1,"selected_cdw_trace_pts")
oid_list = cdw_start_oids
agglines = os.path.join(lscratch, "cdw_trace_results_part1")


#Set up
if arcpy.Exists(agglines):
    print("file is here. I will destroy it for you")
    arcpy.management.Delete(agglines) 
else: 
    print("Feature class doesn't exist")
#select upstream 1 mile
arcpy.env.addOutputsToMap = True
oid_fieldname = arcpy.Describe(cdw_trace_pts1).OIDFieldName
i = 1
list_length = len(oid_list)
error_list_cdw = [] #list of point oid that caused an error

t = time.localtime()
start_time = time.strftime("%H:%M:%S", t)
t1 = time.time()
print(f"start time is {start_time}")
      
for pt in oid_list:
    try:
        where = f"{oid_fieldname} = {pt}"
        start_pt = arcpy.conversion.ExportFeatures(cdw_trace_pts1, os.path.join(lscratch,"temp_point"), where) 
        arcpy.tn.Trace(
            in_trace_network=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\stream_buffers_1.gdb\Stream_buffers\CDW_streams_network",
            trace_type="UPSTREAM",
            starting_points= start_pt,
            barriers="TN_Temp_Barriers",
            path_direction="NO_DIRECTION",
            shortest_path_network_attribute_name="",
            include_barriers="INCLUDE_BARRIERS",
            validate_consistency="VALIDATE_CONSISTENCY",
            ignore_barriers_at_starting_points="DO_NOT_IGNORE_BARRIERS_AT_STARTING_POINTS",
            allow_indeterminate_flow="IGNORE_INDETERMINATE_FLOW",
            condition_barriers=None,
            function_barriers="ADD Segment_Length IS_GREATER_THAN_OR_EQUAL_TO 5280 true",
            traversability_scope="BOTH_JUNCTIONS_AND_EDGES",
            functions=None,
            output_conditions=None,
            result_types="AGGREGATED_GEOMETRY",
            selection_type="NEW_SELECTION",
            clear_all_previous_trace_results="DO_NOT_CLEAR_ALL_PREVIOUS_TRACE_RESULTS",
            trace_name="",
            #aggregated_points=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\RiparianZones_20240109.gdb\Trace_Results_Aggregated_Points",
            aggregated_lines= agglines, 
            out_network_layer=None
        )
    except:
        print(f"{pt} failed")
        error_list_cdw.append(pt)
    finally:
        print(f"{i} of {list_length} complete")
        i = i+1
print(error_list_cdw)

t = time.localtime()
end_time = time.strftime("%H:%M:%S", t)
print(end_time)
t2 = time.time()
elapsed_time = ((t2 - t1)/60)
print(f"loop took {elapsed_time} minutes to run")

file is here. I will destroy it for you
start time is 20:54:09
1 of 1419 complete
2 of 1419 complete
3 of 1419 complete
4 of 1419 complete
5 of 1419 complete
6 of 1419 complete
7 of 1419 complete
8 of 1419 complete
9 of 1419 complete
10 of 1419 complete
11 of 1419 complete
12 of 1419 complete
13 of 1419 complete
14 of 1419 complete
15 of 1419 complete
16 of 1419 complete
17 of 1419 complete
18 of 1419 complete
19 of 1419 complete
20 of 1419 complete
21 of 1419 complete
22 of 1419 complete
23 of 1419 complete
24 of 1419 complete
25 of 1419 complete
26 of 1419 complete
27 of 1419 complete
28 of 1419 complete
29 of 1419 complete
30 of 1419 complete
31 of 1419 complete
32 of 1419 complete
33 of 1419 complete
34 of 1419 complete
35 of 1419 complete
36 of 1419 complete
37 of 1419 complete
38 of 1419 complete
39 of 1419 complete
40 of 1419 complete
41 failed
41 of 1419 complete
42 of 1419 complete
43 of 1419 complete
44 of 1419 complete
45 of 1419 complete
46 of 1419 complete
47 of 1419 compl

770 of 1419 complete
771 of 1419 complete
772 of 1419 complete
773 of 1419 complete
774 of 1419 complete
775 of 1419 complete
776 of 1419 complete
777 of 1419 complete
778 of 1419 complete
779 of 1419 complete
780 of 1419 complete
781 of 1419 complete
782 of 1419 complete
783 of 1419 complete
784 of 1419 complete
785 of 1419 complete
786 of 1419 complete
787 of 1419 complete
788 of 1419 complete
789 of 1419 complete
790 of 1419 complete
791 of 1419 complete
792 of 1419 complete
793 of 1419 complete
794 of 1419 complete
795 of 1419 complete
796 of 1419 complete
797 of 1419 complete
798 of 1419 complete
799 of 1419 complete
800 of 1419 complete
801 of 1419 complete
802 of 1419 complete
803 of 1419 complete
804 of 1419 complete
805 of 1419 complete
806 of 1419 complete
807 of 1419 complete
808 of 1419 complete
809 of 1419 complete
810 of 1419 complete
811 of 1419 complete
812 of 1419 complete
813 of 1419 complete
814 of 1419 complete
815 of 1419 complete
816 of 1419 complete
817 of 1419 c

In [6]:
error_oids = [41, 73, 75, 96, 128, 130, 132, 172, 191, 292, 384, 385, 405, 523, 564, 565, 593, 599, 616, 621, 645, 663, 686, 769, 840, 877, 906, 907, 965, 1007, 1013, 1034, 1045, 1136, 1143, 1179, 1182, 1183, 1185, 1186, 1189, 1190, 1191, 1192, 1193, 1194, 1195, 1203, 1209, 1229]
len(error_oids)

50

In [19]:
arcpy.management.Merge([os.path.join(home, 'CDW_Trace_results_partbeween_upstream'), os.path.join(lscratch, "cdw_trace_results_part1")],
                       os.path.join(scratch1,"cdw_trace_results_merged"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_merged'>

In [20]:
#Merge results and erase from cdw upstream erase lines to get remaining streams
arcpy.analysis.Erase(os.path.join(scratch1,"non_cdw_buffer_diss"), 
                     os.path.join(scratch1,"cdw_trace_results_merged"), 
                     os.path.join(scratch1, "cdw_trace_results_erase"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_erase'>

In [21]:
arcpy.management.MultipartToSinglepart(in_features = os.path.join(scratch1, "cdw_trace_results_erase"),
                                       out_feature_class = os.path.join(scratch1, "cdw_trace_erase_SP"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_erase_SP'>

In [13]:
arcpy.management.FeatureVerticesToPoints(os.path.join(lscratch, "cdw_trace_results_part1"), os.path.join(scratch1, "cdw_trace_results_bothpts"), 'BOTH_ENDS')
#Select remaining upstream non-cdw lines that intersect with end point (or just not start point maybe) of part1 results


<Result 'cdw_trace_erase_lyr'>

In [22]:
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "cdw_trace_erase_SP"), "cdw_trace_erase_lyr")
arcpy.management.SelectLayerByLocation("cdw_trace_erase_lyr", 'INTERSECT', os.path.join(scratch1,"cdw_trace_results_merged"))
arcpy.management.SelectLayerByLocation("cdw_trace_erase_lyr", 'INTERSECT', os.path.join(scratch1, "cdw_trace_results_bothpts"), selection_type = "SUBSET_SELECTION", invert_spatial_relationship = "INVERT")

<Result 'cdw_trace_erase_lyr'>

In [24]:
arcpy.management.CopyFeatures("cdw_trace_erase_lyr", os.path.join(scratch1,"cdw_trace_results_missed"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_missed'>

In [ ]:
#For some reason the upstream trace did not always get to the desired length, so some are trace short. 

#merge cdw_trace_results_missed with other results
#erase merge results from non-cdw
#buffer dissolve the remaining non cdw erase segments
#select from upstream merge any segments that are < 5280
#make starting points or just make a layer from them
#Select non-cdw that intersect starting points and are < ? 5280? 2640? Trial and error

In [3]:
arcpy.management.Merge([os.path.join(scratch1,"cdw_trace_results_merged"), os.path.join(scratch1,"cdw_trace_results_missed")],
                       os.path.join(scratch1, "cdw_trace_results2")
                       )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results2'>

In [5]:
#Merge results and erase from cdw upstream erase lines to get remaining streams
arcpy.analysis.Erase(os.path.join(scratch1,"non_cdw_buffer_diss"), 
                     os.path.join(scratch1,"cdw_trace_results2"), 
                     os.path.join(scratch1, "cdw_trace_results_erase2"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_erase2'>

In [6]:
#Making individual mutipart features from any connected portions of a linear network
#https://gis.stackexchange.com/questions/174752/dissolve-a-polyline-feature-class-so-that-touching-features-dissolve-into-a-sing

mp_input = os.path.join(scratch1, "cdw_trace_results_erase2")#Multipart polylines
target_lines = os.path.join(scratch1, "cdw_upstream_erase_SP") #Singlepart target lines
field_name = "transfer_code"

arcpy.management.MultipartToSinglepart(mp_input, target_lines)

if len(arcpy.ListFields(target_lines, field_name)) == 0:
    print("Field does not exist")       
else:
    arcpy.management.DeleteField(target_lines, field_name)
    
buffer = arcpy.analysis.Buffer(target_lines, os.path.join(scratch1,"non_cdw_buffer"), "1 Feet", "FULL", "ROUND", "ALL", None, "PLANAR")
buffer1 = arcpy.management.MultipartToSinglepart(os.path.join(scratch1,"non_cdw_buffer"), os.path.join(scratch1,"cdw_upstream_buffer_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, field_name, "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(target_lines, 
                                         buffer1, 
                                         os.path.join(scratch1,"non_cdw_buffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL")
non_cdw_diss1 = arcpy.management.Dissolve(buffer_join, 
                                        os.path.join(scratch1, "non_cdw_buffer_diss1"), 
                                        field_name, 
                                        None, 
                                        "MULTI_PART", 
                                        "DISSOLVE_LINES", '')

In [24]:
#Making individual mutipart features from any connected portions of a linear network
#https://gis.stackexchange.com/questions/174752/dissolve-a-polyline-feature-class-so-that-touching-features-dissolve-into-a-sing

mp_input = os.path.join(scratch1, "cdw_trace_results2")#Multipart polylines
target_lines = os.path.join(scratch1, "cdw_upstream_SP1") #Singlepart target lines
field_name = "transfer_code"

arcpy.management.MultipartToSinglepart(mp_input, target_lines)

if len(arcpy.ListFields(target_lines, field_name)) == 0:
    print("Field does not exist")       
else:
    arcpy.management.DeleteField(target_lines, field_name)
    
buffer = arcpy.analysis.Buffer(target_lines, os.path.join(scratch1,"cdw_upbuffer"), "1 Feet", "FULL", "ROUND", "ALL", None, "PLANAR")
buffer1 = arcpy.management.MultipartToSinglepart(os.path.join(scratch1,"cdw_upbuffer"), os.path.join(scratch1,"cdw_upbuffer_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, field_name, "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(target_lines, 
                                         buffer1, 
                                         os.path.join(scratch1,"cdw_upbuffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL")
cdw_up_diss = arcpy.management.Dissolve(buffer_join, 
                                        os.path.join(scratch1, "cdw_upbuffer_diss"), 
                                        field_name, 
                                        None, 
                                        "MULTI_PART", 
                                        "DISSOLVE_LINES", '')

In [29]:
cdw_upresults = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "cdw_upbuffer_diss"), "cdw_upstream", where_clause = "Shape_Length < 5280")

In [25]:
#Select start points that DO NOT intersect the between/upstream lines already seleced
cdw_upresults = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "cdw_upbuffer_diss"), "cdw_upstream", where_clause = "Shape_Length < 5280")

arcpy.management.SelectLayerByLocation(cdw_upresults, 
                                       overlap_type = "INTERSECT", 
                                       select_features = non_cdw_diss1, 
                                       #search_distance, 
                                       selection_type = "SUBSET_SELECTION"
                                       #invert_spatial_relationship
                                      )
cdw__sel = arcpy.management.MakeFeatureLayer(cdw_upresults, "cdw_up_sel")

In [26]:
arcpy.management.FeatureVerticesToPoints(cdw__sel, os.path.join(scratch1, "temp_end_pts"), 'END')

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\temp_end_pts'>

In [35]:
non_cdw_erase = arcpy.management.MakeFeatureLayer(non_cdw_diss1, "non_cdw_erase")

In [36]:
arcpy.management.SelectLayerByLocation(non_cdw_erase, 
                                       overlap_type = "INTERSECT", 
                                       select_features = cdw__sel, 
                                       #search_distance, 
                                       selection_type = "SUBSET_SELECTION"
                                       #invert_spatial_relationship
                                      )

arcpy.management.SelectLayerByLocation(non_cdw_erase, 
                                       overlap_type = "INTERSECT", 
                                       select_features = os.path.join(scratch1, "temp_end_pts"), 
                                       #search_distance, 
                                       selection_type = "SUBSET_SELECTION",
                                       invert_spatial_relationship = "INVERT"
                                      )


<Result 'non_cdw_erase'>

In [33]:
arcpy.conversion.ExportFeatures(non_cdw_erase, 
                                os.path.join(scratch1,"cdw_trace_results_missed1"),
                                where_clause = "Shape_Length < 3200")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_missed1'>

In [38]:
arcpy.management.CopyFeatures(non_cdw_erase, os.path.join(scratch1, "non_cdw_long_strags"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\non_cdw_long_strags'>

In [39]:
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "non_cdw_long_strags"), 
                                  "non_cdw_erase_long", 
                                  where_clause = "Shape_Length > 3200")
#Manual checking and edits on these

<Result 'non_cdw_erase_long'>

In [40]:
arcpy.management.CopyFeatures("non_cdw_erase_long", 
                              os.path.join(scratch1,"cdw_trace_results_missed2"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\cdw_trace_results_missed2'>

In [41]:
arcpy.management.Merge([os.path.join(scratch1, "cdw_trace_results2"), os.path.join(scratch1,"cdw_trace_results_missed1"), os.path.join(scratch1,"cdw_trace_results_missed2")],
                       os.path.join(home, "cdw_trace_results3")
                       )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\cdw_trace_results3'>

In [42]:
arcpy.management.Dissolve(
    in_features=os.path.join(home, "cdw_trace_results3"),
    out_feature_class= os.path.join(home, "CDW_1mile_upstream_final"),
    dissolve_field=None,
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\CDW_1mile_upstream_final'>

## Code Streams

In [6]:
# streams to merge
nj2015_temp2 = os.path.join(home, "stream_regs_with_exclusions_v2")

c1_upstream = os.path.join(home, "C1_upstream_final")
tm_upstream = os.path.join(home, "TM_1mile_upstream_final")
cdw_upstream = os.path.join(home, "CDW_1mile_upstream_final")

In [ ]:
# Align upstream lines with nhd network due to edits in network
#arcpy.edit.AlignFeatures(tm_upstream, nj2015_temp2, 50)
#arcpy.edit.AlignFeatures(cdw_upstream, nj2015_temp2, 50)

In [30]:
arcpy.management.FeatureVerticesToPoints(c1_upstream, os.path.join(scratch1, "c1_up_pts"), "BOTH_ENDS")
arcpy.management.FeatureVerticesToPoints(tm_upstream, os.path.join(scratch1, "tm_up_pts"), "BOTH_ENDS")
arcpy.management.FeatureVerticesToPoints(cdw_upstream, os.path.join(scratch1, "cdw_up_pts"), "BOTH_ENDS")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\tm_up_pts'>

In [32]:
arcpy.management.Merge([os.path.join(scratch1,"c1_up_pts"),os.path.join(scratch1,"tm_up_pts"),os.path.join(scratch1,"cdw_up_pts")],
                      os.path.join(scratch1, "up_ends_merge"))
arcpy.management.DeleteIdentical(os.path.join(scratch1, "up_ends_merge"), "Shape")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\up_ends_merge'>

In [34]:
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "up_ends_merge"), 
                                       os.path.join(scratch1, "up_ends_merge_SP"))

#snap intersected points to nhd points
arcpy.edit.Snap(os.path.join(scratch1, "up_ends_merge_SP"), 
                [[nj2015_temp2, 'EDGE', '10 feet']])

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\up_ends_merge_SP'>

In [44]:
arcpy.management.SplitLineAtPoint(nj2015_temp2, 
                                  os.path.join(scratch1, "up_ends_merge"), 
                                  os.path.join(home, "stream_regs_with_exclusions_v3"),
                                  '5 feet')

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\stream_regs_with_exclusions_v3'>

In [45]:
arcpy.management.MakeFeatureLayer(os.path.join(home, "stream_regs_with_exclusions_v3"), "nhd_lines")

<Result 'nhd_lines'>

In [46]:
#Add upstream columns
arcpy.management.SelectLayerByLocation("nhd_lines", 
                                       "SHARE_A_LINE_SEGMENT_WITH", 
                                       c1_upstream
                                      )

input_fc = os.path.join(home, "stream_regs_with_exclusions_v3")
field_name = "C1_upstream"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("nhd_lines",["OID@"])]
oidfield = arcpy.Describe("nhd_lines").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[1] = 0
        curs.updateRow(row)
del curs

In [47]:
#Add upstream columns
arcpy.management.SelectLayerByLocation("nhd_lines", 
                                       "SHARE_A_LINE_SEGMENT_WITH", 
                                       tm_upstream
                                      )

input_fc = os.path.join(home, "stream_regs_with_exclusions_v3")
field_name = "TM_upstream"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("nhd_lines",["OID@"])]
oidfield = arcpy.Describe("nhd_lines").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[1] = 0
        curs.updateRow(row)
del curs

In [48]:
#Add upstream columns
arcpy.management.SelectLayerByLocation("nhd_lines", 
                                       "SHARE_A_LINE_SEGMENT_WITH", 
                                       cdw_upstream
                                      )

input_fc = os.path.join(home, "stream_regs_with_exclusions_v3")
field_name = "CDW_upstream"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("nhd_lines",["OID@"])]
oidfield = arcpy.Describe("nhd_lines").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[1] = 0
        curs.updateRow(row)
del curs

# Clean up and finishing steps

## Assembling Background Layers

In [18]:
nj2015_swqs = os.path.join(home,"nhd_swqs_join")

#Export a layer for each category
arcpy.conversion.ExportFeatures(nj2015_swqs, os.path.join(blayers, "Streams_TroutProduction"), where_clause = "TP = 1")
arcpy.conversion.ExportFeatures(nj2015_swqs, os.path.join(blayers, "Streams_TroutMaintenance"), where_clause = "TM = 1")
arcpy.conversion.ExportFeatures(nj2015_swqs, os.path.join(blayers, "Streams_C1"), where_clause = "C1 = 1")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\CategoryOneStreams'>

In [21]:
# Landscape Project CDW streams
LP_all = os.path.join(newgdb, "Landscape_proj_all_cdw")
arcpy.conversion.ExportFeatures(LP_all, os.path.join(blayers, "LP_FHASCDW"), where_clause = "FHA_SCDW = 'Yes'")
arcpy.management.CopyFeatures(os.path.join(home, "CDW_all_regions"), os.path.join(blayers, "LP_FHASCDW_inclusive"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\LP_FHASCDW_inclusive'>

In [43]:
arcpy.conversion.ExportFeatures(os.path.join(home, "nhd_cdw_split1"), 
                               os.path.join(blayers, "CDW_All_Features_LP"),
                                where_clause = "all_cdw = 1")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\All_SCDW_Features'>

In [26]:
# Unregulated LULC and canals
lulc = r"X:\projects\njwrap\Data\NJDEP_LULUC\Land_lu_2015.gdb\Land_lu_2015"
arcpy.conversion.ExportFeatures(lulc, 
                                os.path.join(blayers, "UnregulatedLULC"), 
                                where_clause = "LU15 = 5120 Or LU15 = 7310 Or LU15 = 7320 Or LU15 = 7330 Or LU15 = 5430 Or LU15 = 6111 Or LU15 = 6112 Or LU15 = 6130 OR LU15 = 6141"
                               )

arcpy.conversion.ExportFeatures(os.path.join(home, "stream_regs_with_exclusions_v2"),
                                os.path.join(blayers, "UnregulatedCanals"),
                                where_clause = "Canal = 1"
                               )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\Riparian_Zone_Input_Layers.gdb\\UnregulatedCanals'>

## Add some extra field information

In [ ]:
#Label Segments that aren't in SWQS
    #Erase SWQS-aligned from nj2015
    #Intersect to create points
    #Split at points
    #Select areas that don't share a segment/not identical (test results)
    #Label selected areas as outside of SWQS mapping 

In [ ]:
#Add possibly unregulated reason from LULC that are treated as unregulated but also from LULC and nhd that aren't exluded: stormwater basins, pipelines

In [41]:
#1499 (Stormwater Basins)
lulc = r"X:\projects\njwrap\Data\NJDEP_LULUC\Land_lu_2015.gdb\Land_lu_2015"
splitting_polygon2 = arcpy.management.MakeFeatureLayer(lulc, 
                                                       "sw_basin", 
                                                       where_clause = "LU15 = 1499")

In [ ]:
#Label segments that are in pipelines and probably not regulated
#FTYPE = 428"

## Adding buffer width columns

### Coding unregulated

In [15]:
streams_ed = os.path.join(home, "stream_regs_with_exclusions_v3")

In [18]:
#LULC excluded classes--identify them so that there is no upstream trace from theres

#5120 (Canals) 
#7310, 7320, 7330 (Stone Quarries, Sand and Gravel Pits (Borrow Pits), Other Mining)
#5430 (Atlantic Ocean) 
#5420 (Dredged Lagoon)
#6111      Saline Marsh (Low marsh) 
#6112      Saline Marsh (High marsh) 
#6130      Vegetated Dune Communities 
#6141      Phragmites Dominate Coastal Wetlands 


nj2020_lulc = arcpy.management.MakeFeatureLayer(r"https://services1.arcgis.com/QWdNfRs7lkPq4g4Q/arcgis/rest/services/Land_Use_2020/FeatureServer/5", "lulc2020")
unregulatedlulc = arcpy.management.MakeFeatureLayer(nj2020_lulc, 
                                                       "lulc_unregulated", 
                                                       where_clause = "LU15 = 5120 Or LU15 = 7310 Or LU15 = 7320 Or LU15 = 7330 Or LU15 = 5430 Or LU15 = 6111 Or LU15 = 6112 Or LU15 = 6130 OR LU15 = 6141"
                                                       )

arcpy.management.Dissolve("lulc_unregulated", os.path.join(home,"lulc_exclusions_diss"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\lulc_exclusions_diss'>

In [19]:
#Add exclusion field for LULC
input_lines = streams_ed
splitting_polygon = os.path.join(home,"lulc_exclusions_diss")
field_name = "regulated"
output =  os.path.join(home, "stream_regs_with_exclusions_v4")

split_by_polygon_zero(input_lines, splitting_polygon, field_name, output)

Field exists


### Labeling canals

In [20]:
buffer2 = arcpy.analysis.Buffer(os.path.join(scratch1, "canals_PolygonToCenterline"), 
                                os.path.join(scratch1,"temp_buffer2"), 
                                "80 Feet", 
                                "FULL", 
                                "FLAT", "ALL", None, "PLANAR")                        

In [21]:
arcpy.management.MakeFeatureLayer(os.path.join(home, "stream_regs_with_exclusions_v4"), "lyr_lines1")

arcpy.management.SelectLayerByLocation("lyr_lines1", 
                                       "WITHIN", 
                                       buffer2)
arcpy.management.SelectLayerByAttribute("lyr_lines1",
                                        selection_type = "ADD_TO_SELECTION",
                                        where_clause = "GNIS_NAME LIKE '%Canal'")

<Result 'lyr_lines1'>

In [22]:
input_fc = os.path.join(home, "stream_regs_with_exclusions_v4")
field_name = "regulated"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("lyr_lines1",["OID@"])]
oidfield = arcpy.Describe("lyr_lines1").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 0
        else:
            row[0] = 1 
        curs.updateRow(row)
del curs

Field exists


In [23]:
input_fc = os.path.join(home, "stream_regs_with_exclusions_v4")
field_name = "Canal"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.AddField_management(input_fc, field_name, "SHORT")
else:
    print("Field exists")

#Make a list from the selected features
oidList = [oid[0] for oid in arcpy.da.SearchCursor("lyr_lines1",["OID@"])]
oidfield = arcpy.Describe("lyr_lines1").OIDFieldName
fields = [oidfield, field_name]

#Code features in the list 
with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 1
        else:
            row[0] = 0 
        curs.updateRow(row)
del curs

Field exists


## Clean Up

In [24]:
streams_ed = os.path.join(home, "stream_regs_with_exclusions_v4")

In [25]:
#Clean up some of the extraneous fields
# Use ListFields to get a list of field objects
fieldObjList = arcpy.ListFields(streams_ed, "ORIG*")

# Create an empty list that will be populated with field names        
fieldNameList = []

# For each field in the object list, add the field name to the
# name list. Exclude required fields to prevent errors
for field in fieldObjList:
    if not field.required:
        fieldNameList.append(field.name)

In [26]:
fieldNameList

['ORIG_FID', 'ORIG_SEQ']

In [27]:
arcpy.management.DeleteField(in_table = streams_ed, 
                             drop_field = fieldNameList, 
                             method = "DELETE_FIELDS")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\stream_regs_with_exclusions_v4'>

In [28]:
# Add field to for each buffer width
if len(arcpy.ListFields(streams_ed, "buffer_width_300")) == 0:
         arcpy.AddField_management(streams_ed, "buffer_width_300", "SHORT")
else:
    print("Field exists")

#300' Buffer
fields = ['C1', 'C1_upstream', 'buffer_width_300']  

with arcpy.da.UpdateCursor(streams_ed, fields) as cursor:
    for row in cursor:
        if row[0] + row[1] > 0:
            row[2] = 1
        else:
            row[2] = 0
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

Field exists


In [29]:
#150' buffer
if len(arcpy.ListFields(streams_ed, "buffer_width_150")) == 0:
         arcpy.AddField_management(streams_ed, "buffer_width_150", "SHORT")
else:
    print("Field exists")

fields1 = ['TP', 'TP_upstream', 'TM', 'TM_upstream', 'all_cdw', 'CDW_upstream', 'buffer_width_150']
with arcpy.da.UpdateCursor(streams_ed, fields1) as cursor:
    for row in cursor:
        res_list = [x for x in row[0:6]]
        result = sum(res_list)
        if result > 0:
            row[6] = 1
        else:
            row[6] = 0
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor


Field exists


In [30]:
#50 buffer
if len(arcpy.ListFields(streams_ed, "buffer_width_50")) == 0:
         arcpy.AddField_management(streams_ed, "buffer_width_50", "SHORT")
else:
    print("Field exists")

fields2 = ['buffer_width_300', 'buffer_width_150', 'buffer_width_50']

with arcpy.da.UpdateCursor(streams_ed, fields2) as cursor:
    for row in cursor:
        if row[0] + row[1] > 0:
            row[2] = 0
        else:
            row[2] = 1
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

Field exists


In [31]:
#Widest buffer

arcpy.management.AddField(streams_ed, "widest_buffer_width", 'SHORT')
fields3 = ['buffer_width_300', 'buffer_width_150', 'regulated', 'widest_buffer_width']

with arcpy.da.UpdateCursor(streams_ed, fields3) as cursor:
    for row in cursor:
        if row [2] == 0:
            row[3] = 0
        elif row[0] == 1:
            row[3] = 300
        elif row[1] == 1:
            row[3] = 150
        else:        
            row[3] = 50
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

In [32]:
timestr = time.strftime("%Y%m%d_%H%M%S")
print(timestr)
arcpy.management.CopyFeatures(streams_ed, os.path.join(home, f"FHA_RiparianZones_{timestr}"), '', None, None, None)

20240308_160120


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_RiparianZones_20240308_160120'>

In [33]:
streams_ed = os.path.join(home, f"FHA_RiparianZones_{timestr}")

In [34]:
# Create a list of keeper fields and delete the rest
#If these terms are in the field names, keep the field
keeper_txt = ['cdw', 'C1', 'c1', 'TP', 'TM', 'CDW', 'buffer', 'regulated', 'GNIS', 'Canal']

# Add all fields from inputs.
flist = [f.name for f in arcpy.ListFields(streams_ed)]

# Name fields you want. Could get these names programmatically too.
keepers = [f for f in flist if(any(ele in f for ele in keeper_txt))]

keepers

['GNIS_NAME', 'C1', 'TP', 'TM', 'all_cdw', 'regulated', 'Canal', 'TP_upstream', 'C1_upstream', 'TM_upstream', 'CDW_upstream', 'buffer_width_300', 'buffer_width_150', 'buffer_width_50', 'widest_buffer_width']

In [35]:
#Delete fields
arcpy.management.DeleteField(streams_ed, keepers, "KEEP_FIELDS")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_RiparianZones_20240308_160120'>

In [36]:
field_names = [f.name for f in arcpy.ListFields(streams_ed, field_type= 'SmallInteger')]
field_names.remove('widest_buffer_width')
print(field_names)

['C1', 'TP', 'TM', 'all_cdw', 'regulated', 'Canal', 'TP_upstream', 'C1_upstream', 'TM_upstream', 'CDW_upstream', 'buffer_width_300', 'buffer_width_150', 'buffer_width_50']


In [37]:
#Translate fields to more user-friendly yes/no instead of 1/0
#Calculate field if oldfield = 1, return 'Yes'

#New field, string
for field in field_names:
    new_field = f"{field}_a"
    arcpy.management.AddField(streams_ed, new_field, 'TEXT')
    fields2 = [field, new_field]
    
    with arcpy.da.UpdateCursor(streams_ed, fields2) as cursor:
        for row in cursor:
            if row[0] == 1:
                row[1] = "Yes"
            else:
                row[1] = "No"
# Update the cursor with the updated list
            cursor.updateRow(row)
    del cursor

In [38]:
#Fields clean up from making new text fields
arcpy.management.DeleteField(streams_ed, field_names)

#update field names
fieldList = arcpy.ListFields(streams_ed)
for field in fieldList:
    if field.name.endswith('_a'):
        new_field = field.name[:-2]
        arcpy.management.AlterField(streams_ed, field.name, new_field)

In [39]:
field_names = [f.name for f in arcpy.ListFields(streams_ed)]
field_names

['OBJECTID', 'SHAPE', 'GNIS_NAME', 'widest_buffer_width', 'SHAPE_Length', 'C1', 'TP', 'TM', 'all_cdw', 'regulated', 'Canal', 'TP_upstream', 'C1_upstream', 'TM_upstream', 'CDW_upstream', 'buffer_width_300', 'buffer_width_150', 'buffer_width_50']

In [40]:
# make things mutually exclusive:

with arcpy.da.UpdateCursor(streams_ed, field_names) as cursor:
    for row in cursor:
        if row[5] == 'Yes': #C1
            row[12] = 'No'  #C1 upstream
        elif row[6] == 'Yes':
            row[11] = 'No'
        elif row [7] == 'Yes':
            row[13] = 'No'
        elif row[8] == 'Yes':        
            row[14] = 'No'
# Update the cursor with the updated list
        cursor.updateRow(row)
del cursor

In [41]:
#Reorder fields. Could do above when I translate from 0/1 to yes/no
messy_input_file = streams_ed
#Get the field names in your file
field_names = [f.name for f in arcpy.ListFields(messy_input_file)]
print(field_names)

['OBJECTID', 'SHAPE', 'GNIS_NAME', 'widest_buffer_width', 'SHAPE_Length', 'C1', 'TP', 'TM', 'all_cdw', 'regulated', 'Canal', 'TP_upstream', 'C1_upstream', 'TM_upstream', 'CDW_upstream', 'buffer_width_300', 'buffer_width_150', 'buffer_width_50']


In [45]:
#Reorder them however you want. Just include the fields you want to keep. Keep OBJECTID but don't keep any of the other standard fields (Shape, Shape_Length, Shape_Area)
new_field_order = ['OBJECTID', 'widest_buffer_width', 'GNIS_NAME', 'regulated', 'Canal',  'C1', 'TP', 'TM', 'all_cdw', 'C1_upstream', 'TP_upstream', 'TM_upstream', 'CDW_upstream', 'buffer_width_300', 'buffer_width_150', 'buffer_width_50']

In [46]:
#parameters
timestr = time.strftime("%Y%m%d_%H%M%S")

messy_input_file = streams_ed
out_path = home
out_name = f"FHA_RiparianZones_{timestr}"
#new_field_order = 
reorder_streams = os.path.join(home, f"FHA_RiparianZones_{timestr}") 

reorderfields(messy_input_file, out_path, out_name, new_field_order)

In [47]:
reorder_streams = os.path.join(home, f"FHA_RiparianZones_{timestr}") 

#Clean up of fields, rename and give aliases
arcpy.management.AlterField(reorder_streams, 'all_cdw', new_field_name = 'FHA_SCDW', new_field_alias = "FHA_SCDW")
arcpy.management.AlterField(reorder_streams, 'CDW_upstream', new_field_name = 'FHA_SCDW_upstream', new_field_alias = "Drains to E&T habitat stream")

#arcpy.management.AlterField(reorder_streams, 'FHA_SCDW', new_field_alias = "FHA_SCDW")
#arcpy.management.AlterField(reorder_streams, 'FHA_SCDW_upstream', new_field_alias = "Drains to E&T habitat stream")


arcpy.management.AlterField(reorder_streams, 'buffer_width_300', new_field_alias = "300 foot buffer")
arcpy.management.AlterField(reorder_streams, 'buffer_width_150', new_field_alias = '150 foot buffer')
arcpy.management.AlterField(reorder_streams, 'buffer_width_50', new_field_alias = '50 foot buffer')
arcpy.management.AlterField(reorder_streams, 'widest_buffer_width', new_field_alias = 'Widest Applicable Buffer')
arcpy.management.AlterField(reorder_streams, "C1", '', "Category One")
arcpy.management.AlterField(reorder_streams, "TP", '', "Trout Production")
arcpy.management.AlterField(reorder_streams, "TM", '', "Trout Maintenance")
arcpy.management.AlterField(reorder_streams, 'regulated', new_field_alias = "Regulated by FHA")
arcpy.management.AlterField(reorder_streams, 'Canal', new_field_alias = "Unregulated Canal")

#Alias the upstreams
arcpy.management.AlterField(reorder_streams, "c1_upstream", '', "Drains to C1 stream")
arcpy.management.AlterField(reorder_streams, "TP_upstream", '', "Drains to Trout Production stream")
arcpy.management.AlterField(reorder_streams, "TM_upstream", '', "Drains to Trout Maintenance stream")

#Shape field
arcpy.management.AlterField(reorder_streams, "Shape_Length", '', "Lenth (ft)")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_RiparianZones_20240308_160842'>

In [49]:
print(f"FHA_RiparianZones_{timestr}")

FHA_RiparianZones_20240308_160842


# Creating buffer polygons

## Stream water bodies

In [4]:
streams = arcpy.management.MakeFeatureLayer(os.path.join(home, "FHA_RiparianZones_20240308_160842"), "stream_classes_lyr") #, "FTYPE <> 566") #exclude coastlines
nj2015_polys = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_waterbody'

### Non-Delaware River

In [5]:
#Stream polys without the delware river. Run Delware separately because it needs to be wider and has some other issues
stream_polysND = arcpy.management.MakeFeatureLayer(nj2015_polys, "stream_polys", "FTYPE = 460 And GNIS_NAME <> 'Delaware River'") #Include estuaries? FTYPE 493

In [55]:
#Select only stream polys (not ponds/lakes/estuaries) and corresponding centerlines
#if arcpy.Exists(os.path.join(scratch1, "stream_polys_Dissolve1")):
#    print("File exists")
#else:
arcpy.management.Dissolve(stream_polysND, 
                        os.path.join(scratch1, "stream_polys_Dissolve1"), 
                        dissolve_field = ['GNIS_NAME'],
                        multi_part = "SINGLE_PART"
                             )
    
stream_polys_diss = os.path.join(scratch1, "stream_polys_Dissolve1")

In [6]:
stream_polys_diss = os.path.join(scratch1, "stream_polys_Dissolve1")

In [56]:
arcpy.analysis.Clip(streams, stream_polys_diss, os.path.join(scratch1, "streampolys_clip"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\streampolys_clip'>

In [ ]:
# #Get list of unique values from a field
# def unique_values(table, field):
#     with arcpy.da.SearchCursor(table, [field]) as cursor:
#         return sorted({row[0] for row in cursor})

In [12]:
# #Add upstream columns
# stream_clips = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "streampolys_clip"), "stream_clips")

# input_fc = stream_clips
# field_name = "PolygonNameMatch"

# if len(arcpy.ListFields(input_fc, field_name)) == 0:
#          arcpy.AddField_management(input_fc, field_name, "SHORT")
# else:
#     print("Field exists")

# #Make a list of GNIS names for polygons
# sp_list = unique_values(stream_polys_diss, "GNIS_NAME")
# fields = ["GNIS_Name", field_name]

# #Code features in the list 
# with arcpy.da.UpdateCursor(input_fc, fields) as curs:
#     for row in curs:
#         if row[0] in sp_list: 
#             row[1] = 1
#         else:
#             row[1] = 0
#         curs.updateRow(row)
# del curs

# #select streams in list 
# arcpy.management.SelectLayerByAttribute(stream_clips, where_clause = "PolygonNameMatch = 1") #, {invert_where_clause})

# arcpy.management.CopyFeatures(stream_clips, os.path.join(scratch1, "streampolyNameMatch"))

In [57]:
buffer = arcpy.analysis.Buffer(os.path.join(scratch1, "streampolys_clip"), os.path.join(scratch1, "streams_temp_buffer"), "1 Feet", "FULL", "ROUND", "LIST", ['widest_buffer_width'])
buffer1 = arcpy.management.MultipartToSinglepart(buffer, os.path.join(scratch1, "temp_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, "transfer_code", "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "streampolys_clip"), buffer1, os.path.join(scratch1, "clip_streams_by_width_ND"), "JOIN_ONE_TO_ONE", "KEEP_ALL", match_option = "WITHIN")
temp1 = arcpy.management.Dissolve(buffer_join, os.path.join(scratch1, "clip_streams_diss_by_width_ND"), ["transfer_code", "widest_buffer_width"], None, "MULTI_PART", "DISSOLVE_LINES", '')

In [30]:
long_streams1 = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "clip_streams_diss_by_width_ND"), "long_streams_only", "Shape_Length > 700")

In [31]:
arcpy.management.SelectLayerByLocation(streams, "SHARE_A_LINE_SEGMENT_WITH", long_streams1)
arcpy.management.CopyFeatures(streams, os.path.join(scratch1, "stream_poly_streams"))
arcpy.analysis.Clip(os.path.join(scratch1, "stream_poly_streams"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "stream_poly_streams_clip"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_poly_streams_clip'>

In [32]:
arcpy.analysis.Clip(os.path.join(scratch1, "stream_poly_streams"), stream_polys_diss, os.path.join(scratch1, "stream_poly_streams_clip"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_poly_streams_clip'>

In [33]:
long_streams = os.path.join(scratch1, "stream_poly_streams_clip")

In [18]:
arcpy.management.MakeFeatureLayer(long_streams, "300ft_buffer", "widest_buffer_width = 300")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "300ft_buffer",
    out_feature_class=os.path.join(scratch1,"Transects_300ft"),
    interval="30000 Feet",
    transect_length="500 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_300ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip300"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip300"), os.path.join(scratch1, "transects_clip300_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip300_SP"), "transects_300")
arcpy.management.MakeFeatureLayer(long_streams, "non_300", "widest_buffer_width <> 300")
arcpy.management.SelectLayerByLocation("transects_300", "INTERSECT", "300ft_buffer")
arcpy.management.SelectLayerByLocation("transects_300", "INTERSECT", "non_300", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_300", os.path.join(scratch1, "transects300_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects300_transitions'>

In [19]:
arcpy.management.MakeFeatureLayer(long_streams, "150ft_buffer", "widest_buffer_width = 150")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "150ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_150ft"),
    interval="30000 Feet",
    transect_length="900 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_150ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip150"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip150"), os.path.join(scratch1, "transects_clip150_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip150_SP"), "transects_150")
arcpy.management.MakeFeatureLayer(long_streams, "non_150", "widest_buffer_width <> 150")
arcpy.management.SelectLayerByLocation("transects_150", "INTERSECT", "150ft_buffer")
arcpy.management.SelectLayerByLocation("transects_150", "INTERSECT", "non_150", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_150", os.path.join(scratch1, "transects150_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects150_transitions'>

In [20]:
arcpy.management.MakeFeatureLayer(long_streams, "50ft_buffer", "widest_buffer_width = 50")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "50ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_50ft"),
    interval="30000 Feet",
    transect_length="900 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_50ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip50"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip50"), os.path.join(scratch1, "transects_clip50_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip50_SP"), "transects_50")
arcpy.management.MakeFeatureLayer(long_streams, "non_50", "widest_buffer_width <> 50")
arcpy.management.SelectLayerByLocation("transects_50", "INTERSECT", "50ft_buffer")
arcpy.management.SelectLayerByLocation("transects_50", "INTERSECT", "non_50", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_50", os.path.join(scratch1, "transects50_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects50_transitions'>

In [21]:
arcpy.management.MakeFeatureLayer(long_streams, "0ft_buffer", "widest_buffer_width = 0")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "0ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_0ft"),
    interval="30000 Feet",
    transect_length="900 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_0ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip0"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip0"), os.path.join(scratch1, "transects_clip0_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip0_SP"), "transects_0")
arcpy.management.MakeFeatureLayer(long_streams, "non_0", "widest_buffer_width <> 0")
arcpy.management.SelectLayerByLocation("transects_0", "INTERSECT", "0ft_buffer")
arcpy.management.SelectLayerByLocation("transects_0", "INTERSECT", "non_0", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_0", os.path.join(scratch1, "transects0_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects0_transitions'>

In [22]:
arcpy.management.Merge([os.path.join(scratch1, "transects300_transitions"), os.path.join(scratch1, "transects150_transitions"), os.path.join(scratch1, "transects50_transitions"), os.path.join(scratch1, "transects0_transitions")], 
                      os.path.join(scratch1, "transect_transitions_merge"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transect_transitions_merge'>

In [ ]:
#Manual edits on Manasquan River, Rancocas Creek, Back Channel, Flat Creek, Swimming River, Mullica

In [38]:
#Subdivide waterbodies by transects:
arcpy.management.FeatureToPolygon([stream_polys_diss, os.path.join(scratch1, "transect_transitions_merge")], 
                                  os.path.join(scratch1, "stream_polys_temp_split"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_polys_temp_split'>

In [39]:
#That process filled spaces that were not meant to be water
arcpy.analysis.Erase(os.path.join(scratch1, "stream_polys_temp_split"), 
                     os.path.join(scratch1, "stream_polys_Dissolve1"),
                     os.path.join(scratch1, "filled_holes")
                    )

arcpy.analysis.Erase(os.path.join(scratch1, "stream_polys_temp_split"), 
                     os.path.join(scratch1, "filled_holes"),
                     os.path.join(scratch1, "stream_polys_split1")
                    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_polys_split1'>

In [40]:
#Split stream lines at all polygon divisions, both preexisting in the dataset and from the transects
arcpy.analysis.Intersect([os.path.join(scratch1, "stream_polys_split1"), long_streams1], 
                         os.path.join(scratch1, "temp_split_pts"), 
                         output_type = "POINT")

arcpy.management.SplitLineAtPoint(long_streams1, 
                                  os.path.join(scratch1, "temp_split_pts"),
                                  os.path.join(scratch1, "split_streams_for_buff"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_buff'>

In [41]:
arcpy.management.Sort(os.path.join(scratch1, "split_streams_for_buff"), 
                      os.path.join(scratch1, "split_streams_for_buff_sort"), 
                      [["Shape_Length", "DESCENDING"]])


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_buff_sort'>

In [42]:
poly_buffwidth = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "stream_polys_split1"),
                                            os.path.join(scratch1, "split_streams_for_buff_sort"), 
                                            os.path.join(home,"stream_polys_buffwidth_join"), 
                                            match_option = "CONTAINS")

In [47]:
#polygons with buffer widths joined
spolys = arcpy.management.MakeFeatureLayer(os.path.join(home,"stream_polys_buffwidth_join"), "spolys")

In [56]:
#Fill nulls with values of adjacent polygons
widths = [300, 150, 50, 0]

for i in widths:
    arcpy.management.MakeFeatureLayer(spolys, "spoly_nulls", where_clause = "widest_buffer_width IS NULL")
    arcpy.management.MakeFeatureLayer(spolys, f"{i}_buff", where_clause = f"widest_buffer_width = {i}")
    arcpy.management.SelectLayerByLocation("spoly_nulls", "BOUNDARY_TOUCHES", f"{i}_buff")
    
    input_fc = os.path.join(home,"stream_polys_buffwidth_join")
    field_name = "widest_buffer_width"

    #Make a list from the selected features
    oidList = [oid[0] for oid in arcpy.da.SearchCursor("spoly_nulls",["OID@"])]
    oidfield = arcpy.Describe(input_fc).OIDFieldName
    fields = [oidfield, field_name]

    #Code features in the list 
    with arcpy.da.UpdateCursor(input_fc, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = f"{i}"
            curs.updateRow(row)
    del curs
    

In [96]:
#Split lines at points
arcpy.management.SplitLineAtPoint(os.path.join(scratch1, "stream_lake_clip"), 
                                  os.path.join(scratch1, "lake_split_pts"), 
                                  os.path.join(scratch1, "streams_lake_split"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\streams_lake_split'>

### Delaware River

In [57]:
#Stream polys without the delware river. Run Delware separately because it needs to be wider and has some other issues
stream_polysD = arcpy.management.MakeFeatureLayer(nj2015_polys, "stream_polys", "FTYPE = 460 And GNIS_NAME = 'Delaware River'") #Include estuaries? FTYPE 493

In [58]:
#Select only stream polys (not ponds/lakes/estuaries) and corresponding centerlines
#if arcpy.Exists(os.path.join(scratch1, "stream_polys_Dissolve1")):
#    print("File exists")
#else:
arcpy.management.Dissolve(stream_polysD, 
                        os.path.join(scratch1, "stream_polys_DissolveD"), 
                        dissolve_field = ['GNIS_NAME'],
                        multi_part = "SINGLE_PART"
                             )
    
stream_polys_diss = os.path.join(scratch1, "stream_polys_DissolveD")

In [6]:
stream_polys_diss = os.path.join(scratch1, "stream_polys_DissolveD")

In [59]:
arcpy.analysis.Clip(streams, stream_polys_diss, os.path.join(scratch1, "streampolys_clip"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\streampolys_clip'>

In [61]:
buffer = arcpy.analysis.Buffer(os.path.join(scratch1, "streampolys_clip"), os.path.join(scratch1, "streams_temp_buffer"), "1 Feet", "FULL", "ROUND", "LIST", ['widest_buffer_width'])
buffer1 = arcpy.management.MultipartToSinglepart(buffer, os.path.join(scratch1, "temp_MultipartToSi1"))
arcpy.management.CalculateField(buffer1, "transfer_code", "!OBJECTID!", "PYTHON3", '', "LONG", "NO_ENFORCE_DOMAINS")
buffer_join = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "streampolys_clip"), buffer1, os.path.join(scratch1, "clip_streams_by_width_D"), "JOIN_ONE_TO_ONE", "KEEP_ALL", match_option = "WITHIN")
temp1 = arcpy.management.Dissolve(buffer_join, os.path.join(scratch1, "clip_streams_diss_by_widthD"), ["transfer_code", "widest_buffer_width"], None, "MULTI_PART", "DISSOLVE_LINES", '')

In [62]:
long_streams1 = arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "clip_streams_diss_by_widthD"), "long_streams_only", "Shape_Length > 700")

In [63]:
arcpy.management.SelectLayerByLocation(streams, "SHARE_A_LINE_SEGMENT_WITH", long_streams1)
arcpy.management.CopyFeatures(streams, os.path.join(scratch1, "stream_poly_streams"))
arcpy.analysis.Clip(os.path.join(scratch1, "stream_poly_streams"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "stream_poly_streams_clipD"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_poly_streams_clipD'>

In [64]:
long_streams = os.path.join(scratch1, "stream_poly_streams_clipD")

In [65]:
arcpy.management.MakeFeatureLayer(long_streams, "300ft_buffer", "widest_buffer_width = 300")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "300ft_buffer",
    out_feature_class=os.path.join(scratch1,"Transects_300ft"),
    interval="30000 Feet",
    transect_length="5000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_300ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip300"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip300"), os.path.join(scratch1, "transects_clip300_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip300_SP"), "transects_300")
arcpy.management.MakeFeatureLayer(long_streams, "non_300", "widest_buffer_width <> 300")
arcpy.management.SelectLayerByLocation("transects_300", "INTERSECT", "300ft_buffer")
arcpy.management.SelectLayerByLocation("transects_300", "INTERSECT", "non_300", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_300", os.path.join(scratch1, "transects300_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects300_transitions'>

In [66]:
arcpy.management.MakeFeatureLayer(long_streams, "150ft_buffer", "widest_buffer_width = 150")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "150ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_150ft"),
    interval="30000 Feet",
    transect_length="5000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_150ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip150"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip150"), os.path.join(scratch1, "transects_clip150_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip150_SP"), "transects_150")
arcpy.management.MakeFeatureLayer(long_streams, "non_150", "widest_buffer_width <> 150")
arcpy.management.SelectLayerByLocation("transects_150", "INTERSECT", "150ft_buffer")
arcpy.management.SelectLayerByLocation("transects_150", "INTERSECT", "non_150", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_150", os.path.join(scratch1, "transects150_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects150_transitions'>

In [67]:
arcpy.management.MakeFeatureLayer(long_streams, "50ft_buffer", "widest_buffer_width = 50")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "50ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_50ft"),
    interval="30000 Feet",
    transect_length="5000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_50ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip50"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip50"), os.path.join(scratch1, "transects_clip50_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip50_SP"), "transects_50")
arcpy.management.MakeFeatureLayer(long_streams, "non_50", "widest_buffer_width <> 50")
arcpy.management.SelectLayerByLocation("transects_50", "INTERSECT", "50ft_buffer")
arcpy.management.SelectLayerByLocation("transects_50", "INTERSECT", "non_50", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_50", os.path.join(scratch1, "transects50_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects50_transitions'>

In [68]:
arcpy.management.MakeFeatureLayer(long_streams, "0ft_buffer", "widest_buffer_width = 0")

arcpy.management.GenerateTransectsAlongLines(
    in_features= "0ft_buffer",
    out_feature_class= os.path.join(scratch1,"Transects_0ft"),
    interval="30000 Feet",
    transect_length="5000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,"Transects_0ft"), 
                    stream_polys_diss, 
                    os.path.join(scratch1, "transects_clip0"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, "transects_clip0"), os.path.join(scratch1, "transects_clip0_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, "transects_clip0_SP"), "transects_0")
arcpy.management.MakeFeatureLayer(long_streams, "non_0", "widest_buffer_width <> 0")
arcpy.management.SelectLayerByLocation("transects_0", "INTERSECT", "0ft_buffer")
arcpy.management.SelectLayerByLocation("transects_0", "INTERSECT", "non_0", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures("transects_0", os.path.join(scratch1, "transects0_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects0_transitions'>

In [69]:
arcpy.management.Merge([os.path.join(scratch1, "transects300_transitions"), os.path.join(scratch1, "transects150_transitions"), os.path.join(scratch1, "transects50_transitions"), os.path.join(scratch1, "transects0_transitions")], 
                      os.path.join(scratch1, "transect_transitions_mergeD"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transect_transitions_mergeD'>

In [ ]:
#Manual edits 

In [85]:
#Subdivide waterbodies by transects:
arcpy.management.FeatureToPolygon([stream_polys_diss, os.path.join(scratch1, "transect_transitions_mergeD")], 
                                  os.path.join(scratch1, "stream_polys_temp_splitD"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_polys_temp_splitD'>

In [86]:
#That process filled spaces that were not meant to be water
arcpy.analysis.Erase(os.path.join(scratch1, "stream_polys_temp_splitD"), 
                     stream_polys_diss,
                     os.path.join(scratch1, "filled_holes")
                    )

arcpy.analysis.Erase(os.path.join(scratch1, "stream_polys_temp_splitD"), 
                     os.path.join(scratch1, "filled_holes"),
                     os.path.join(scratch1, "stream_polys_splitD")
                    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_polys_splitD'>

In [87]:
#Split stream lines at all polygon divisions, both preexisting in the dataset and from the transects
arcpy.analysis.Intersect([os.path.join(scratch1, "stream_polys_splitD"), long_streams1], 
                         os.path.join(scratch1, "temp_split_pts"), 
                         output_type = "POINT")

arcpy.management.SplitLineAtPoint(long_streams1, 
                                  os.path.join(scratch1, "temp_split_pts"),
                                  os.path.join(scratch1, "split_streams_for_buffD"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_buffD'>

In [88]:
arcpy.management.Sort(os.path.join(scratch1, "split_streams_for_buffD"), 
                      os.path.join(scratch1, "split_streams_for_buffD_sort"), 
                      [["Shape_Length", "DESCENDING"]])


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_buffD_sort'>

In [89]:
poly_buffwidth = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "stream_polys_splitD"),
                                            os.path.join(scratch1, "split_streams_for_buffD_sort"), 
                                            os.path.join(home,"stream_polys_buffwidth_joinD"), 
                                            match_option = "CONTAINS")

In [90]:
#polygons with buffer widths joined
spolys = arcpy.management.MakeFeatureLayer(os.path.join(home,"stream_polys_buffwidth_joinD"), "spolys")

In [91]:
#Fill nulls with values of adjacent polygons
widths = [300, 150, 50, 0]

for i in widths:
    arcpy.management.MakeFeatureLayer(spolys, "spoly_nulls", where_clause = "widest_buffer_width IS NULL")
    arcpy.management.MakeFeatureLayer(spolys, f"{i}_buff", where_clause = f"widest_buffer_width = {i}")
    arcpy.management.SelectLayerByLocation("spoly_nulls", "BOUNDARY_TOUCHES", f"{i}_buff")
    
    input_fc = os.path.join(home,"stream_polys_buffwidth_joinD")
    field_name = "widest_buffer_width"

    #Make a list from the selected features
    oidList = [oid[0] for oid in arcpy.da.SearchCursor("spoly_nulls",["OID@"])]
    oidfield = arcpy.Describe(input_fc).OIDFieldName
    fields = [oidfield, field_name]

    #Code features in the list 
    with arcpy.da.UpdateCursor(input_fc, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = f"{i}"
            curs.updateRow(row)
    del curs
    

## Buffer other waterbodies

### Lakes

In [92]:
#Buffer the remaining polygons that aren't streams, canals/ditches, or the ocean
lake_polys = arcpy.management.MakeFeatureLayer(nj2015_polys, "lake_polys", "FTYPE <> 460 And FTYPE <> 336 And FTYPE <> 445 And FTYPE <> 493")

In [93]:
#Clip streams to lakes so that they will be "within"
arcpy.analysis.Clip(streams, lake_polys, os.path.join(scratch1, "stream_lake_clip"))

In [98]:
input_fc = lake_polys
field_name = "transfer_code_lakes"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.management.CalculateField(input_fc,
                                         field_name, 
                                         "!OBJECTID!", 
                                         "PYTHON3", '', 
                                         "LONG", 
                                         "NO_ENFORCE_DOMAINS")
else:
    print("Field exists")

Field exists


In [ ]:
#Make dissolved network of same width
buffer_join = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "streams_lake_clip"),
                                         lake_polys, 
                                         os.path.join(scratch1, "temp_buffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL", 
                                         match_option = "WITHIN")

stream_clip = arcpy.management.Dissolve(os.path.join(scratch1,"temp_buffer_join"), 
                                        os.path.join(scratch1, "lakes_clip_diss"), 
                                        dissolve_field=["widest_buffer_width", "transfer_code_lakes"], 
                                        multi_part= "MULTI_PART")

In [101]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = lake_polys
dis = 300
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects300_transitions'>

In [102]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = lake_polys
dis = 150
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects150_transitions'>

In [103]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = lake_polys
dis = 50
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects50_transitions'>

In [104]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = lake_polys
dis = 0
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects0_transitions'>

In [105]:
arcpy.env.addOutputsToMap = True

arcpy.management.Merge([os.path.join(scratch1, "transects300_transitions"), os.path.join(scratch1, "transects150_transitions"), os.path.join(scratch1, "transects50_transitions"), os.path.join(scratch1, "transects0_transitions")], 
                      os.path.join(scratch1, "transect_transitions_mergeL"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transect_transitions_mergeL'>

In [ ]:
#Manual edits 

In [ ]:
#Subdivide waterbodies by transects:
arcpy.management.FeatureToPolygon(
    in_features="transect_transitions_mergeL;lake_polys",
    out_feature_class= os.path.join(scratch1,"lake_polys_FTP"),
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

In [113]:
#That process filled spaces that were not meant to be water
arcpy.analysis.Erase(os.path.join(scratch1, "lake_polys_FTP"), 
                     lake_polys,
                     os.path.join(scratch1, "filled_holes")
                    )

arcpy.analysis.Erase(os.path.join(scratch1, "lake_polys_FTP"), 
                     os.path.join(scratch1, "filled_holes"),
                     os.path.join(scratch1, "lake_polys_split1")
                    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\lake_polys_split1'>

In [114]:
#Split stream lines at all polygon divisions, both preexisting in the dataset and from the transects
arcpy.analysis.Intersect([os.path.join(scratch1, "lake_polys_split1"), stream_clip], 
                         os.path.join(scratch1, "temp_split_pts"), 
                         output_type = "POINT")

arcpy.management.SplitLineAtPoint(stream_clip, 
                                  os.path.join(scratch1, "temp_split_pts"),
                                  os.path.join(scratch1, "split_streams_for_Lbuff"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_Lbuff'>

In [115]:
arcpy.management.Sort(os.path.join(scratch1, "split_streams_for_Lbuff"), 
                      os.path.join(scratch1, "split_streams_for_Lbuff_sort"), 
                      [["Shape_Length", "DESCENDING"]])


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_Lbuff_sort'>

In [116]:
poly_buffwidth = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "lake_polys_split1"),
                                            os.path.join(scratch1, "split_streams_for_Lbuff_sort"), 
                                            os.path.join(home,"lake_polys_buffwidth_join"), 
                                            match_option = "CONTAINS")

In [117]:
#Fill nulls with values of adjacent polygons
widths = [300, 150, 50, 0]

for i in widths:
    arcpy.management.MakeFeatureLayer(poly_buffwidth, "poly_nulls", where_clause = "widest_buffer_width IS NULL")
    arcpy.management.MakeFeatureLayer(poly_buffwidth, f"{i}_buff", where_clause = f"widest_buffer_width = {i}")
    arcpy.management.SelectLayerByLocation("poly_nulls", "BOUNDARY_TOUCHES", f"{i}_buff")
    
    input_fc = poly_buffwidth
    field_name = "widest_buffer_width"

    #Make a list from the selected features
    oidList = [oid[0] for oid in arcpy.da.SearchCursor("poly_nulls",["OID@"])]
    oidfield = arcpy.Describe(input_fc).OIDFieldName
    fields = [oidfield, field_name]

    #Code features in the list 
    with arcpy.da.UpdateCursor(input_fc, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = f"{i}"
            curs.updateRow(row)
    del curs
    

### Estuaries

In [124]:
#Buffer the remaining polygons that aren't streams, canals/ditches, or the ocean
est_polys = arcpy.management.MakeFeatureLayer(nj2015_polys, "est_polys", "FTYPE = 493")

In [119]:
#Clip streams to lakes so that they will be "within"
arcpy.analysis.Clip(streams, 
                    est_polys, 
                    os.path.join(scratch1, "stream_est_clip"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\stream_est_clip'>

In [120]:
input_fc = est_polys
field_name = "transfer_code_est"

if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.management.CalculateField(input_fc,
                                         field_name, 
                                         "!OBJECTID!", 
                                         "PYTHON3", '', 
                                         "LONG", 
                                         "NO_ENFORCE_DOMAINS")
else:
    print("Field exists")

In [ ]:
#Make dissolved network of same width
buffer_join = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "stream_est_clip"),
                                         est_polys, 
                                         os.path.join(scratch1, "temp_buffer_join"), 
                                         "JOIN_ONE_TO_ONE", 
                                         "KEEP_ALL", 
                                         match_option = "WITHIN")

stream_clip = arcpy.management.Dissolve(os.path.join(scratch1,"temp_buffer_join"), 
                                        os.path.join(scratch1, "ests_clip_diss"), 
                                        dissolve_field=["widest_buffer_width", "transfer_code_est"], 
                                        multi_part= "MULTI_PART")

In [128]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = est_polys
dis = 300
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects300_transitions'>

In [133]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = est_polys
dis = 150
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects150_transitions'>

In [134]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = est_polys
dis = 50
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects50_transitions'>

In [135]:
arcpy.env.addOutputsToMap = False

input_fc = stream_clip
clip_fc = est_polys
dis = 0
####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_buffer",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="2000 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_buffer")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")

arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects0_transitions'>

In [136]:
arcpy.env.addOutputsToMap = True

arcpy.management.Merge([os.path.join(scratch1, "transects300_transitions"), os.path.join(scratch1, "transects150_transitions"), os.path.join(scratch1, "transects50_transitions"), os.path.join(scratch1, "transects0_transitions")], 
                      os.path.join(scratch1, "transect_transitions_mergeE"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transect_transitions_mergeE'>

In [ ]:
#Manual edits

In [139]:
#Subdivide waterbodies by transects:
arcpy.management.FeatureToPolygon(
    in_features= [os.path.join(scratch1, "transect_transitions_mergeE"),est_polys],
    out_feature_class= os.path.join(scratch1,"est_polys_FTP"),
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\est_polys_FTP'>

In [140]:
#That process filled spaces that were not meant to be water
arcpy.analysis.Erase(os.path.join(scratch1, "est_polys_FTP"), 
                     est_polys,
                     os.path.join(scratch1, "filled_holes")
                    )

arcpy.analysis.Erase(os.path.join(scratch1, "est_polys_FTP"), 
                     os.path.join(scratch1, "filled_holes"),
                     os.path.join(scratch1, "est_polys_split1")
                    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\est_polys_split1'>

In [141]:
#Split stream lines at all polygon divisions, both preexisting in the dataset and from the transects
arcpy.analysis.Intersect([os.path.join(scratch1, "est_polys_split1"), stream_clip], 
                         os.path.join(scratch1, "temp_split_pts"), 
                         output_type = "POINT")

arcpy.management.SplitLineAtPoint(stream_clip, 
                                  os.path.join(scratch1, "temp_split_pts"),
                                  os.path.join(scratch1, "split_streams_for_Ebuff"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_Lbuff'>

In [143]:
arcpy.management.SplitLineAtPoint(stream_clip, 
                                  os.path.join(scratch1, "temp_split_pts"),
                                  os.path.join(scratch1, "split_streams_for_Ebuff"), 
                                  "5 feet")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_Ebuff'>

In [144]:
arcpy.management.Sort(os.path.join(scratch1, "split_streams_for_Ebuff"), 
                      os.path.join(scratch1, "split_streams_for_Ebuff_sort"), 
                      [["Shape_Length", "DESCENDING"]])


<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\split_streams_for_Ebuff_sort'>

In [149]:
poly_buffwidth = arcpy.analysis.SpatialJoin(os.path.join(scratch1, "est_polys_split1"),
                                            os.path.join(scratch1, "split_streams_for_Ebuff_sort"), 
                                            os.path.join(home,"est_polys_buffwidth_join"), 
                                            match_option = "CONTAINS")

In [150]:
#Fill nulls with values of adjacent polygons
widths = [300, 150, 50, 0]

for i in widths:
    arcpy.management.MakeFeatureLayer(poly_buffwidth, "poly_nulls", where_clause = "widest_buffer_width IS NULL")
    arcpy.management.MakeFeatureLayer(poly_buffwidth, f"{i}_buff", where_clause = f"widest_buffer_width = {i}")
    arcpy.management.SelectLayerByLocation("poly_nulls", "BOUNDARY_TOUCHES", f"{i}_buff")
    
    input_fc = poly_buffwidth
    field_name = "widest_buffer_width"

    #Make a list from the selected features
    oidList = [oid[0] for oid in arcpy.da.SearchCursor("poly_nulls",["OID@"])]
    oidfield = arcpy.Describe(input_fc).OIDFieldName
    fields = [oidfield, field_name]

    #Code features in the list 
    with arcpy.da.UpdateCursor(input_fc, fields) as curs:
        for row in curs:
            if row[0] in oidList: 
                row[1] = f"{i}"
            curs.updateRow(row)
    del curs
    

In [151]:
#Code features in the list 

oidList = [324, 346, 884]

input_fc = poly_buffwidth
field_name = "widest_buffer_width"
fields = [oidfield, field_name]

with arcpy.da.UpdateCursor(input_fc, fields) as curs:
    for row in curs:
        if row[0] in oidList: 
            row[1] = 50
        curs.updateRow(row)
del curs

## Combine Polygons

In [152]:
#Merge ND, Delaware, Lakes, Estuaries
all_polys = arcpy.management.Merge([os.path.join(home,"stream_polys_buffwidth_join"), os.path.join(home,"stream_polys_buffwidth_joinD"), os.path.join(home,"lake_polys_buffwidth_join"), os.path.join(home,"est_polys_buffwidth_join")], 
                                  os.path.join(scratch1, "all_polybuffwidth_merge"))#classified_polys_merge

In [14]:
all_polys = os.path.join(scratch1, "all_polybuffwidth_merge")

In [27]:
# #Code features in the list 

# oidList = [33610, 33611]

# input_fc = os.path.join(scratch1, "all_polybuffwidth_merge")
# field_name = "widest_buffer_width"
# fields = ['OBJECTID', field_name]

# with arcpy.da.UpdateCursor(input_fc, fields) as curs:
#     for row in curs:
#         if row[0] in oidList: 
#             row[1] = 0
#         curs.updateRow(row)
# del curs

In [28]:
all_poly_diss = arcpy.management.Dissolve(in_features = all_polys, 
                          out_feature_class = os.path.join(home, "all_polybuffwidth_diss"), 
                          dissolve_field = "widest_buffer_width", 
                          multi_part = "SINGLE_PART")

In [29]:
non_null = arcpy.management.MakeFeatureLayer(all_poly_diss, "polys_non_null", where_clause = "widest_buffer_width > 0")

In [30]:
#Buffer and dissolve by buffer width
polys_buff = arcpy.analysis.Buffer(non_null, 
                      os.path.join(scratch1, "Final_buffer_polygons"),
                      buffer_distance_or_field = "widest_buffer_width",
                      dissolve_option = "LIST",
                      dissolve_field = ["widest_buffer_width"]
                     )

## Centerline buffer

In [181]:
streams1 = arcpy.management.MakeFeatureLayer(os.path.join(home, "FHA_RiparianZones_20240308_160842"), "stream_classes_lyr") #exclude coastlines 


In [182]:
#buffer centerlines
centerlines_buff = arcpy.analysis.Buffer("stream_classes_lyr", 
                                         os.path.join(scratch1, "Final_buffer_centerlines"), 
                                         buffer_distance_or_field = "widest_buffer_width", 
                                         line_end_type="ROUND", 
                                         dissolve_option="LIST",
                                         dissolve_field = "widest_buffer_width")

## Combine buffers

In [31]:
polys_buff = os.path.join(scratch1, "Final_buffer_polygons")
centerlines_buff = os.path.join(scratch1, "Final_buffer_centerlines")

In [32]:
#Merge and Dissolve centerline and waterbodies buffers

timestr = time.strftime("%Y%m%d_%H%M%S")
allbuff = arcpy.management.Merge([polys_buff, centerlines_buff], 
                                 os.path.join(home,f"FHA_buffers_merge{timestr}"))


In [33]:
timestr

'20240318_164834'

In [34]:
#Dissolve by buffer width
arcpy.management.Dissolve(allbuff, 
                          os.path.join(home,f"FHA_buffers_{timestr}"),
                          dissolve_field="widest_buffer_width", 
                          multi_part = "SINGLE_PART"
                         )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_20240318_164834'>

### Trim ends?

In [23]:
streams = arcpy.management.MakeFeatureLayer(os.path.join(home, "FHA_RiparianZones_20240308_160842"), "streams_lyr") #, "FTYPE <> 566") #exclude coastlines
buffs = os.path.join(home,f"FHA_buffers_{timestr}")

#nj2015_polys = r'X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\Hydr_NHD_2015.gdb\Hydr_NHD_2015\Hydr_NHD_2015_waterbody'

In [25]:
#arcpy.env.addOutputsToMap = False

input_fc = streams
dis = 300
clip_fc = arcpy.management.MakeFeatureLayer(buffs, f"{dis}ft_buffer", f"widest_buffer_width = {dis}")

####
arcpy.management.MakeFeatureLayer(input_fc, f"{dis}ft_stream", f"widest_buffer_width = {dis}")
end_pts = arcpy.management.FeatureVerticesToPoints(f"{dis}ft_stream", os.path.join(scratch1, "TM_end_pts"), 'END')
buffs_invert = arcpy.management.MakeFeatureLayer(buffs, f"{dis}ft_buffer", f"widest_buffer_width <> {dis}")

arcpy.management.GenerateTransectsAlongLines(
    in_features= f"{dis}ft_stream",
    out_feature_class=os.path.join(scratch1,f"Transects_{dis}ft"),
    interval="30000 Feet",
    transect_length="600 Feet",
    include_ends="END_POINTS"
)

arcpy.analysis.Clip(os.path.join(scratch1,f"Transects_{dis}ft"), 
                    clip_fc, 
                    os.path.join(scratch1, f"transects_clip{dis}"))

#Multipart to singlepart
arcpy.management.MultipartToSinglepart(os.path.join(scratch1, f"transects_clip{dis}"), os.path.join(scratch1, f"transects_clip{dis}_SP"))

#Select and retain only the transect at transitions between riparian widths
arcpy.management.MakeFeatureLayer(os.path.join(scratch1, f"transects_clip{dis}_SP"), f"transects_{dis}")
arcpy.management.MakeFeatureLayer(input_fc, f"non_{dis}", f"widest_buffer_width <> {dis}")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"{dis}ft_stream")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", f"non_{dis}", selection_type = "SUBSET_SELECTION")
arcpy.management.SelectLayerByLocation(f"transects_{dis}", "INTERSECT", end_pts, selection_type = "SUBSET_SELECTION")


arcpy.management.CopyFeatures(f"transects_{dis}", os.path.join(scratch1, f"transects{dis}_transitions"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\scratch1.gdb\\transects300_transitions'>

In [ ]:
#Reconcile conflicts where buffers of different widths overlap
#Create dupe id field
input_fc
field_name = "DuplicateID"
if len(arcpy.ListFields(input_fc, field_name)) == 0:
         arcpy.management.CalculateField(input_fc, field_name, "!OBJECTID!", field_type = "LONG")
else:
    print("Field exists")

#Feature to polygon transects and each width
arcpy.management.FeatureToPolygon(
    in_features="transect_transitions_mergeL;lake_polys",
    out_feature_class= os.path.join(scratch1,"lake_polys_FTP"),
    cluster_tolerance=None,
    attributes="ATTRIBUTES",
    label_features=None
)

#Multipart to singlepart
#Sort by area
#delete identical dupe id field


## Clean up layer

In [35]:
arcpy.management.RepairGeometry(allbuff)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_merge20240318_164834'>

In [36]:
arcpy.analysis.Erase(os.path.join(home,f"FHA_buffers_{timestr}"), 
                    os.path.join(home, "lulc_exclusions_diss"),
                    os.path.join(home,f"FHA_buffers_erase{timestr}")
                    )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_erase20240318_164834'>

In [37]:
print(f"FHA_buffers_erase{timestr}")

FHA_buffers_erase20240318_164834


## Clean up fields and field names

In [38]:
buffs = os.path.join(home,f"FHA_buffers_erase{timestr}")

In [39]:
#Clean up of fields, rename and give aliases
arcpy.management.AlterField(buffs, 'widest_buffer_width', new_field_alias = 'Widest Applicable Buffer')

#Shape field
arcpy.management.AlterField(buffs, "Shape_Length", '', "Lenth (ft)")
arcpy.management.AlterField(buffs, "Shape_Area", '', "Area (sq ft)")

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_erase20240318_164834'>

In [3]:
timestr = "20240318_164834"

In [4]:
arcpy.management.EliminatePolygonPart(
    in_features=os.path.join(home, f"FHA_buffers_erase{timestr}"),
    out_feature_class= os.path.join(home, f"FHA_buffers_erase{timestr}a"),
    condition="AREA",
    part_area="60000 SquareFeetUS",
    part_area_percent=5,
    part_option="CONTAINED_ONLY"
)

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_erase20240318_164834a'>

In [7]:
arcpy.management.RepairGeometry(os.path.join(home, f"FHA_buffers_erase{timestr}a"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_erase20240318_164834a'>

In [8]:
#Dissolve by buffer width
arcpy.management.Dissolve(#buffs, 
                          os.path.join(home, f"FHA_buffers_erase{timestr}a"),
                          os.path.join(home,f"FHA_buffers_{timestr}_dissolved"),
                          multi_part = "SINGLE_PART"
                         )

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\RiparianZones_20240109.gdb\\FHA_buffers_20240318_164834_dissolved'>

In [9]:
# Set local variables
out_folder_path = r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109"
out_name = "FHARiparianZoneMaps_20240319.gdb"
fmaps = os.path.join(out_folder_path, out_name)

if arcpy.Exists(fmaps):
    print("GDB exists")
else: 
    print("Creating GDB")
    # Execute CreateFileGDB
    arcpy.CreateFileGDB_management(out_folder_path, out_name)

GDB exists


In [10]:
#Copy final maps to fresh gdb
fdate = "20240319"

#lines
arcpy.management.CopyFeatures(os.path.join(home, "FHA_RiparianZones_20240308_160842"), os.path.join(fmaps, f"A_FHA_Lines_{fdate}"))

#Polygons
arcpy.management.CopyFeatures(s.path.join(home, "all_polybuffwidth_diss"), os.path.join(fmaps, f"B_FHA_Polygons_{fdate}"))

#Buffers by width
arcpy.management.CopyFeatures(os.path.join(home, f"FHA_buffers_erase{timestr}a"), os.path.join(fmaps, f"C_FHA_Buffers_{fdate}"))

#Buffers Dissolved
arcpy.management.CopyFeatures(os.path.join(home,f"FHA_buffers_{timestr}_dissolved"), os.path.join(fmaps, f"D_FHA_BuffersDissolved_{fdate}"))

<Result 'X:\\projects\\njwrap\\Task_2\\task2a_Regulatory_buffers\\RiparianZones_20240109\\FHARiparianZoneMaps_20240319.gdb\\C_FHA_BuffersDissolved_20240319'>

In [11]:
arcpy.management.ApplySymbologyFromLayer(
    in_layer=os.path.join(fmaps, f"C_FHA_Buffers_{fdate}"),
    in_symbology_layer=r"X:\projects\njwrap\Task_2\task2a_Regulatory_buffers\RiparianZones_20240109\FHA_Buffers_PairedColors.lyrx",
    symbology_fields="VALUE_FIELD widest_buffer_width widest_buffer_width",
    update_symbology="DEFAULT"
)

<Result 'B_FHA_Polygons_20240319'>

# Create visualization
https://loading.io/color/feature/Paired-12/

In [ ]:
#Set symbology
#Share > Save as Layer
#Apply symbology from layer (Data Management Tool)